In [1]:
# import packages
import os
from pyspark import SparkContext, SparkConf
import numpy as np
# import seaborn as sns
# sns.set(rc={'figure.figsize':(15,5)},font_scale=1.2)
# color = ["#e74c3c", "#2ecc71"]
# sns.set_palette(color)
# sns.set_style("whitegrid")
import matplotlib.pyplot as plt

import pandas as pd
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None)


print(os.environ['SPARK_HOME'])
from pyspark.sql import SparkSession, HiveContext, SQLContext
from pyspark.sql.functions import when, desc
from pyspark.sql.types import *
from pyspark.sql import Row, functions as F
from pyspark.sql.functions import to_date, trunc, unix_timestamp, date_format
from pyspark.sql.window import Window
from datetime import datetime
from pyspark.storagelevel import StorageLevel

/databricks/spark

In [2]:
ivr_call_disp_cd_dictionary = spark.read.csv('adl://prodrxperso.azuredatalakestore.net/rxperso/dev/extrnl/ivr_call_disp_cd_dictionary.csv', header = True)

rr_ivr_call_disp_cd_dictionary = ivr_call_disp_cd_dictionary\
                                    .select('ivr_call_disp_cd', 'ivr_file_typ_cd', 'ivr_call_disp_dsc','ivr_gnrl_disp_cd')\
                                    .filter(F.col('ivr_file_typ_cd') == 'RR')

In [3]:
  #rxoppty_script_final data prep
cols = ['rxc_pat_id', 'store_nbr', 'rx_nbr', 'rx_fill_nbr', 'run_dt', 'pgm_cd',
        'sub_pgm_cd', 'con_track_typ', 'dispnd_ndc', 'antcptd_refill_due_dt', 'fill_dt']

rxoppty_target = spark.table('rx_rtl_base.rxoppty_script_final') \
        .withColumn('sub_pgm_cd', \
                    F.when((F.col('sub_pgm_cd') == 'RR-IVR') | (F.col('sub_pgm_cd') == 'RR-CC'), 'RR') \
                    .when(F.col('sub_pgm_cd') == 'RR_ARDD', 'ARDD') \
                    .when(F.col('sub_pgm_cd') == 'RR_ARDD2', 'POST2_ARDD') \
                    .when(F.col('sub_pgm_cd') == 'RR_EXP', 'EXP') \
                    .otherwise(F.col('sub_pgm_cd'))) \
        .filter(F.col('sub_pgm_cd').isin(['RR', 'ARDD', 'POST2_ARDD']))\
        .withColumn('day', date_format('run_dt', 'EEEE')) \
        .filter(F.col('run_dt') >= datetime(2018, 6, 1, 0, 0)) \
        .filter(F.col('run_dt') <= datetime(2019, 1, 31, 0, 0)) \
        .filter(F.col('pgm_cd') == 'RR') \
        .filter(F.col('con_track_typ') == 0) \
        .select(*cols) \
        .distinct() 

rxoppty_target = rxoppty_target\
                  .withColumnRenamed('rxc_pat_id', 'rx_rxc_pat_id')\
                  .repartition(300)\
                  .persist(StorageLevel.MEMORY_ONLY_SER)

# ivr_disposition data prep
cols = ['rxc_pat_id', 'store_nbr', 'rx_nbr', 'rx_fill_nbr', 'ivr_src_file_crt_dt', 'ivr_file_typ_cd',
        'ivr_call_disp_cd', 'ivr_call_disp_dsc', 'ivr_gnrl_disp_cd', 'sub_pgm_cd', 'channel', 'disp_flg', 
        'outreach_typ_cd','mobile_number','ivr_refl_req_cd']

rx_disp = spark.table('aoscore_stg.ivr_disposition') \
        .withColumn('sub_pgm_cd', \
                    F.when((F.col('outreach_typ_cd') == 'Fill0RTS') |
                           (F.col('outreach_typ_cd') == 'RTS_RR'), 'Fill0RTS') \
                    .otherwise(F.col('outreach_typ_cd'))) \
        .withColumn('channel', F.when(F.col('ivr_call_disp_cd').like('%SMS%'), 'SMS') \
                    .when(F.col('ivr_call_disp_cd').like('%CON01%'), 'SMS')\
                    .when(F.col('ivr_call_disp_cd').isNull(), 'Not Outreached') \
                    .when(F.col('ivr_call_disp_cd').like('B%'), 'Bypassed').otherwise('IVR')) \
        .withColumn('disp_flg', F.when(F.col('ivr_call_disp_cd').substr(0, 1) == 'B', 'Bypassed') \
                    .when(F.col('ivr_call_disp_cd').isNull(), 'Not Sent') \
                    .otherwise('Outreached')) \
        .filter(F.col('ivr_src_file_crt_dt') >= datetime(2018, 6, 1, 0, 0)) \
        .filter(F.col('ivr_src_file_crt_dt') <= datetime(2019, 1, 31, 0, 0)) \
        .filter(F.col('ivr_file_typ_cd') == 'RR') \
        .withColumn('mobile_number', F.concat(F.lit('001'), F.col('phone_nbr')))\
        .filter(F.col('sub_pgm_cd').isin(['RR','ARDD','POST2_ARDD'])) \
        .join(rr_ivr_call_disp_cd_dictionary, ['ivr_file_typ_cd','ivr_call_disp_cd'], 'left') \
        .drop(rr_ivr_call_disp_cd_dictionary.ivr_file_typ_cd) \
        .drop(rr_ivr_call_disp_cd_dictionary.ivr_call_disp_cd) \
        .select(*cols) \
        .distinct() \
        .filter(F.col('disp_flg') == 'Outreached')



In [4]:
rx_disp = spark.read.parquet('/opt/prod/common/rx_personal/users/c067715/rx_disp')

In [5]:
rx_disp.show(2)

+----------+---------+------+-----------+-------------------+---------------+----------------+--------------------+----------------+----------+-------+----------+---------------+-------------+---------------+
rxc_pat_id|store_nbr|rx_nbr|rx_fill_nbr|ivr_src_file_crt_dt|ivr_file_typ_cd|ivr_call_disp_cd| ivr_call_disp_dsc|ivr_gnrl_disp_cd|sub_pgm_cd|channel| disp_flg|outreach_typ_cd|mobile_number|ivr_refl_req_cd|
+----------+---------+------+-----------+-------------------+---------------+----------------+--------------------+----------------+----------+-------+----------+---------------+-------------+---------------+
1254627667| 7336|781939| 1|2018-09-11 00:00:00| RR| A|Connection, call ...| Err|POST2_ARDD| IVR|Outreached| POST2_ARDD|0014047756595| null|
8699648741| 6998|985789| 2|2018-08-05 00:00:00| RR| SMS| null| null| RR| SMS|Outreached| RR|0013175783989| null|
+----------+---------+------+-----------+-------------------+---------------+----------------+--------------------+----------------+----------+-------+----------+---------------+-------------+---------------+
only showing top 2 rows

In [6]:
# window functions 
w = Window.partitionBy('store_nbr', 'rx_nbr', 'rx_fill_nbr').orderBy(F.col('run_dt'))
multi_rx_w = Window.partitionBy('disp_rxc_pat_id', 'run_dt').orderBy()

# mark NA as NULL to show in ptnt_response_seq
rx_disp = rx_disp.na.fill('NULL')
rx_disp = rx_disp.withColumnRenamed('ivr_src_file_crt_dt', 'run_dt')\
                    .withColumnRenamed('rxc_pat_id','disp_rxc_pat_id')\
                    .withColumn('ivr_refl_req_cd', F.when(F.col('ivr_refl_req_cd').isNull(),'NULL').otherwise(F.col('ivr_refl_req_cd')))\
                    .withColumn('ptnt_response_seq', 
                                F.concat_ws(", ", F.collect_list(F.col('ivr_refl_req_cd')).over(w))) \
                    .withColumn('attempt_seq', 
                                F.concat_ws(", ", F.collect_list(F.col('sub_pgm_cd')).over(w))) \
                    .withColumn('channel_seq', 
                                F.concat_ws(", ", F.collect_list(F.col('channel')).over(w))) \
                    .withColumn('rx_unique', 
                                F.concat('store_nbr', 'rx_nbr', 'rx_fill_nbr')) \
                    .withColumn('num_rx', F.size(F.collect_set('rx_unique').over(multi_rx_w)))\
                    .withColumn('multi_rx_flg', F.when(F.col('num_rx')>1, 1).otherwise(0))


# rr_target joining oppty, disp and drug
df_rx_disp_join_rxoppty = rx_disp \
                            .join(rxoppty_target, ['store_nbr', 'rx_nbr', 'rx_fill_nbr',
                                                   'run_dt', 'sub_pgm_cd'], 'inner') \
                            .withColumnRenamed('dispnd_ndc', 'ndc') \
                            .withColumn('rxc_pat_id', F.coalesce(F.col('disp_rxc_pat_id'), F.col('rx_rxc_pat_id'))) \
                            .drop('disp_rxc_pat_id', 'rx_rxc_pat_id')

drug_df = spark.read.parquet('adl://prodrxperso.azuredatalakestore.net/rxperso/dev/dl/dl_drug')
drug_df = drug_df \
            .select('pas_drug_gcn', 'ndc_final', 'gpi4_drug_class') \
            .withColumnRenamed('ndc_final', 'ndc') \
            .withColumnRenamed('pas_drug_gcn', 'gcn')

# join ivr_rxoppty with drug table for rr_target
ivr_rxoppty_drug = df_rx_disp_join_rxoppty.join(drug_df, 'ndc', 'inner')

w = Window.partitionBy('store_nbr', 'rx_nbr', 'rx_fill_nbr').orderBy(F.col('run_dt'))
w_o = Window.partitionBy('store_nbr', 'rx_nbr', 'rx_fill_nbr', 'disp_flg').orderBy(F.col('run_dt'))

ivr_rxoppty_drug_unique = ivr_rxoppty_drug \
        .select('disp_flg', 'store_nbr', 'rx_nbr', 'rx_fill_nbr', 'run_dt', 'sub_pgm_cd', 'ivr_call_disp_cd') \
        .withColumn('rr_max_run_dt_outreached', F.when(F.col('disp_flg') == 'Outreached', F.max('run_dt').over(w_o))) \
        .withColumn('rr_first_run_dt_outreached', F.when(F.col('disp_flg') == 'Outreached', F.min('run_dt').over(w_o))) \
        .withColumn('rr_first_sub_pgm_cd_outreached', F.when(F.col('disp_flg') == 'Outreached', F.first('sub_pgm_cd').over(w_o))) \
        .withColumn('rr_attempt_nbr_outreached', F.when(F.col('disp_flg') == 'Outreached', 
                                                        (F.sum(F.lit(1)).over(w_o)).cast("decimal(1,0)"))) \
        .withColumn('rr_max_run_dt', F.max('run_dt').over(w)) \
        .withColumn('rr_first_run_dt', (F.min('run_dt').over(w))) \
        .withColumn('rr_attempt_nbr', (F.sum(F.lit(1)).over(w)).cast("decimal(1,0)"))

ivr_rxoppty_drug_v1 = ivr_rxoppty_drug \
                                .join(ivr_rxoppty_drug_unique,
                                      ['store_nbr', 'rx_nbr', 'rx_fill_nbr', 'run_dt',
                                       'sub_pgm_cd', 'ivr_call_disp_cd', 'disp_flg'], 'left') 

ivr_rxoppty_drug_grp_max_atmpt_nbr = ivr_rxoppty_drug_v1 \
                                                .groupBy('store_nbr', 'rx_nbr', 'rx_fill_nbr') \
                                                .agg(F.max('rr_attempt_nbr').alias('rr_max_attempt_nbr'),
                                                     F.max('run_dt').alias('rr_max_run_dt'))\
                                                .repartition(300)\
                                                .persist(StorageLevel.MEMORY_ONLY_SER)

ivr_rxoppty_drug_v1\
    .repartition(400)\
    .write.parquet('/opt/prod/common/rx_personal/users/c083365/ivr_rxoppty_drug_v1', mode = 'overwrite')

ivr_rxoppty_drug_v1 = spark.read.parquet('/opt/prod/common/rx_personal/users/c083365/ivr_rxoppty_drug_v1')
ivr_rxoppty_drug_v2 = ivr_rxoppty_drug_v1 \
                                    .join(ivr_rxoppty_drug_grp_max_atmpt_nbr,
                                          ['store_nbr', 'rx_nbr', 'rx_fill_nbr'], 'left') \
                                    .drop(ivr_rxoppty_drug_v1.rr_attempt_nbr) \
                                    .drop(ivr_rxoppty_drug_v1.rr_max_run_dt)

wo = Window.partitionBy('store_nbr', 'rx_nbr', 'rx_fill_nbr').orderBy(F.col('rr_max_run_dt_outreached').desc())
w = Window.partitionBy('store_nbr', 'rx_nbr', 'rx_fill_nbr').orderBy(F.col('rr_max_run_dt').desc())

ivr_rxoppty_drug_v1_latest = ivr_rxoppty_drug_v2 \
                                    .withColumn('rowNum',  (F.when((F.col('disp_flg') == 'Outreached'), 
                                                                   F.row_number().over(wo)))) \
                                    .filter((F.col('rowNum') == 1))

w1 = Window.partitionBy('store_nbr', 'rx_nbr','rx_fill_nbr').orderBy(F.col('disp_flg').desc())
ivr_rxoppty_drug_v1_latest = ivr_rxoppty_drug_v1_latest.withColumn('rowNum2',(F.row_number().over(w1)))\
                                                            .filter((F.col('rowNum2') == 1))


def prefix_columns(prefix, df):
    l = df.columns
    cols = [prefix + c for c in l]
    return df.toDF(*cols)

ivr_rxoppty_drug_v1_latest = prefix_columns('o_', ivr_rxoppty_drug_v1_latest)

ivr_rxoppty_drug_v1_latest\
    .repartition(400)\
    .write.parquet('/opt/prod/common/rx_personal/users/c083365/ivr_rxoppty_drug_v1_latest', mode = 'overwrite')

curr_join_drug = spark.read.parquet('adl://prodrxperso.azuredatalakestore.net/rxperso/dev/dl/dl_rxfill_base') \
                        .filter((F.col('fill_dt') >= datetime(2018, 6, 1, 0, 0))) \
                        .filter((F.col('pkup_dt') >= datetime(2018, 6, 1, 0, 0)) | (F.col('pkup_dt').isNull())) \
                        .filter((F.col('pkup_dt') >= F.col('fill_dt')) | (F.col('pkup_dt').isNull()))

store_rts = spark.read.csv('adl://prodrxperso.azuredatalakestore.net/rxperso/dev/extrnl/store_configuration.csv', header = True)\
                     .select('store_nbr', 'rts_days_in_bin_nbr')

join_con_rts = [ivr_rxoppty_drug_v1_latest.o_store_nbr == store_rts.store_nbr]
ivr_oppty_final = ivr_rxoppty_drug_v1_latest.join(store_rts, join_con_rts, 'left') \
                                     .drop('store_nbr')
                                
# joining prescription/oppty/disp table
join_cond = [ivr_oppty_final.o_rxc_pat_id == curr_join_drug.rxc_pat_id,
             ivr_oppty_final.o_gcn == curr_join_drug.pas_drug_gcn,
             ivr_oppty_final.o_gpi4_drug_class == curr_join_drug.gpi4_drug_class]

oppty_to_script = ivr_oppty_final \
            .join(curr_join_drug, join_cond, 'inner') \
            .withColumn('RR_target_filter', (F.col('o_rr_max_run_dt_outreached') <= F.col('fill_dt'))) \
            .filter((F.col('RR_target_filter') == 1)) \
            .filter(F.col('fill_dt') <= F.date_add(F.to_date(F.col('o_antcptd_refill_due_dt')), 24)) \
            .withColumn('rr_time_to_fill_after_reached',   F.datediff(F.to_date(F.col('fill_dt')),  
                                                                      F.to_date(F.col('o_rr_max_run_dt_outreached'))))\
            .withColumn('rr_time_to_pickup_after_fill', F.datediff(F.to_date(F.col('pkup_dt')), 
                                                                   F.to_date(F.col('fill_dt')))) \
            .withColumn('rr_target', F.when(((F.col('rr_time_to_pickup_after_fill') <= F.col('rts_days_in_bin_nbr')) &
                                           (F.col('rr_time_to_pickup_after_fill').isNotNull())), 1).otherwise(0)) \
            .withColumn('rr_time_to_pickup_after_reached',F.datediff(F.to_date(F.col('pkup_dt')), 
                                                                     F.to_date(F.col('o_rr_max_run_dt_outreached')))) \
            .withColumn('rr_time_to_pickup_rel_ardd',F.datediff(F.to_date(F.col('pkup_dt')), 
                                                                F.to_date(F.col('o_antcptd_refill_due_dt')))) \
            .withColumn('rr_time_to_fill_rel_ardd', F.datediff(F.to_date(F.col('fill_dt')), 
                                                               F.to_date(F.col('o_antcptd_refill_due_dt'))))


w = Window.partitionBy('o_store_nbr', 'o_rx_nbr', 'o_rx_fill_nbr')\
            .orderBy(F.col('fill_dt'), F.col('pkup_dt'))

df_1 = oppty_to_script \
                .withColumn('rowNum_dups', F.row_number().over(w)) \
                .filter(F.col('rowNum_dups') == 1)\
                .drop('rowNum_dups')

df_2 = oppty_to_script \
                .groupBy('o_store_nbr', 'o_rx_nbr', 'o_rx_fill_nbr') \
                .agg(F.sum(F.lit(1)).alias('n_records_for_target_idenf'))


oppty_to_script_final_v0 = df_1.join(df_2, ['o_store_nbr', 'o_rx_nbr', 'o_rx_fill_nbr'], 'inner')\
                                   .select('o_store_nbr', 'o_rx_nbr', 'o_rx_fill_nbr',
                                           'n_records_for_target_idenf')
oppty_to_script_final = df_1.join(oppty_to_script_final_v0, ['o_store_nbr', 'o_rx_nbr', 'o_rx_fill_nbr'], 
                                  'inner')  
                                
# join back the target script to the ivr opps final table
oppty_to_script_unique_rx = oppty_to_script_final.select('o_store_nbr','o_rx_nbr','o_rx_fill_nbr','ndc','rxc_pat_id',
                                                         'store_nbr','rx_nbr','rx_fill_nbr','pkup_dt','fill_dt',
                                                         'gpi4_drug_class','rr_time_to_fill_after_reached',
                                                         'rr_time_to_pickup_after_fill',
                                                         'rr_time_to_pickup_after_reached',
                                                         'rr_time_to_pickup_rel_ardd','rr_time_to_fill_rel_ardd',
                                                         'rr_target','n_records_for_target_idenf')

df_final = ivr_rxoppty_drug_v1_latest.join(oppty_to_script_unique_rx, ['o_store_nbr', 'o_rx_nbr', 'o_rx_fill_nbr'], 'left')
df_final = df_final \
                .withColumn('rr_target', F.when(F.col('rr_target').isNull(), 0).otherwise(F.col('rr_target'))) \
                .repartition(400)    

df_final.repartition(300).write.parquet('/opt/prod/common/rx_personal/users/c083365/df_final', mode = 'overwrite')

In [7]:
print rx_disp.count()

158322330

In [8]:
df_final.printSchema()

root
-- o_store_nbr: decimal(6,0) (nullable = true)
-- o_rx_nbr: decimal(8,0) (nullable = true)
-- o_rx_fill_nbr: decimal(3,0) (nullable = true)
-- o_run_dt: timestamp (nullable = true)
-- o_sub_pgm_cd: string (nullable = true)
-- o_ivr_call_disp_cd: string (nullable = true)
-- o_disp_flg: string (nullable = true)
-- o_ndc: string (nullable = true)
-- o_ivr_file_typ_cd: string (nullable = true)
-- o_ivr_call_disp_dsc: string (nullable = true)
-- o_ivr_gnrl_disp_cd: string (nullable = true)
-- o_channel: string (nullable = true)
-- o_outreach_typ_cd: string (nullable = true)
-- o_mobile_number: string (nullable = true)
-- o_ivr_refl_req_cd: string (nullable = true)
-- o_ptnt_response_seq: string (nullable = true)
-- o_attempt_seq: string (nullable = true)
-- o_channel_seq: string (nullable = true)
-- o_rx_unique: string (nullable = true)
-- o_num_rx: integer (nullable = true)
-- o_multi_rx_flg: integer (nullable = true)
-- o_pgm_cd: string (nullable = true)
-- o_con_track_typ: string (nullable = true)
-- o_antcptd_refill_due_dt: timestamp (nullable = true)
-- o_fill_dt: timestamp (nullable = true)
-- o_rxc_pat_id: decimal(20,0) (nullable = true)
-- o_gcn: decimal(5,0) (nullable = true)
-- o_gpi4_drug_class: string (nullable = true)
-- o_rr_max_run_dt_outreached: timestamp (nullable = true)
-- o_rr_first_run_dt_outreached: timestamp (nullable = true)
-- o_rr_first_sub_pgm_cd_outreached: string (nullable = true)
-- o_rr_attempt_nbr_outreached: decimal(1,0) (nullable = true)
-- o_rr_first_run_dt: timestamp (nullable = true)
-- o_rr_max_attempt_nbr: decimal(1,0) (nullable = true)
-- o_rr_max_run_dt: timestamp (nullable = true)
-- o_rowNum: integer (nullable = true)
-- o_rowNum2: integer (nullable = true)
-- ndc: string (nullable = true)
-- rxc_pat_id: decimal(18,0) (nullable = true)
-- store_nbr: integer (nullable = true)
-- rx_nbr: decimal(18,0) (nullable = true)
-- rx_fill_nbr: integer (nullable = true)
-- pkup_dt: timestamp (nullable = true)
-- fill_dt: date (nullable = true)
-- gpi4_drug_class: string (nullable = true)
-- rr_time_to_fill_after_reached: integer (nullable = true)
-- rr_time_to_pickup_after_fill: integer (nullable = true)
-- rr_time_to_pickup_after_reached: integer (nullable = true)
-- rr_time_to_pickup_rel_ardd: integer (nullable = true)
-- rr_time_to_fill_rel_ardd: integer (nullable = true)
-- rr_target: integer (nullable = true)
-- n_records_for_target_idenf: long (nullable = true)

In [9]:
df_final = df_final\
            .withColumn('less_than_3_attempt_flg', F.when(F.col('o_rr_attempt_nbr_outreached')<=3,1).otherwise(0))\

In [10]:
df_final = df_final\
            .withColumn('less_than_3_attempt_flg', F.when(F.col('o_rr_attempt_nbr_outreached')<=3,1).otherwise(0))\
            .withColumn('valid_fill_flg', F.when(F.col('rr_time_to_fill_rel_ardd')<= 24, 1).otherwise(0))\
            .withColumn('first_response', F.when((F.col('o_rr_attempt_nbr_outreached')>=2) & 
                                                (F.col('o_ptnt_response_seq').like('N,%')), 'N')\
                                           .when((F.col('o_rr_attempt_nbr_outreached')>=2) & 
                                                (F.col('o_ptnt_response_seq').like('NULL,%')), 'NULL')\
                                           .when((F.col('o_rr_attempt_nbr_outreached')>=2) & 
                                                (F.col('o_ptnt_response_seq').like('Y,%')), 'Y')\
                                           .when((F.col('o_rr_attempt_nbr_outreached')>=2) & 
                                                (F.col('o_ptnt_response_seq').like('S,%')), 'S')\
                                           .otherwise(F.col('o_ptnt_response_seq')))\
            .withColumn('last_response', F.when((F.col('o_rr_attempt_nbr_outreached')>=2) & 
                                                (F.col('o_ptnt_response_seq').like('%, N')), 'N')\
                                           .when((F.col('o_rr_attempt_nbr_outreached')>=2) & 
                                                 (F.col('o_ptnt_response_seq').like('%, NULL')), 'NULL')\
                                           .when((F.col('o_rr_attempt_nbr_outreached')>=2) & 
                                                 (F.col('o_ptnt_response_seq').like('%, Y')), 'Y')\
                                           .when((F.col('o_rr_attempt_nbr_outreached')>=2) & 
                                                 (F.col('o_ptnt_response_seq').like('%, S')), 'S')\
                                           .otherwise(F.col('o_ptnt_response_seq')))\
            .withColumn('first_pgm', F.when((F.col('o_rr_attempt_nbr_outreached')>=2) & 
                                                (F.col('o_attempt_seq').like('ARDD,%')), 'ARDD')\
                                           .when((F.col('o_rr_attempt_nbr_outreached')>=2) & 
                                                (F.col('o_attempt_seq').like('RR,%')), 'RR')\
                                           .when((F.col('o_rr_attempt_nbr_outreached')>=2) & 
                                                (F.col('o_attempt_seq').like('POST2_ARDD,%')), 'POST2_ARDD')\
                                           .otherwise(F.col('o_attempt_seq')))\
            .withColumn('last_pgm', F.when((F.col('o_rr_attempt_nbr_outreached')>=2) & 
                                                (F.col('o_attempt_seq').like('%, ARDD')), 'ARDD')\
                                           .when((F.col('o_rr_attempt_nbr_outreached')>=2) & 
                                                (F.col('o_attempt_seq').like('%, RR')), 'RR')\
                                           .when((F.col('o_rr_attempt_nbr_outreached')>=2) & 
                                                (F.col('o_attempt_seq').like('%, POST2_ARDD')), 'POST2_ARDD')\
                                           .otherwise(F.col('o_attempt_seq')))\
            .withColumn('first_channel', F.when((F.col('o_rr_attempt_nbr_outreached')>=2) & 
                                                (F.col('o_channel_seq').like('IVR,%')), 'IVR')\
                                           .when((F.col('o_rr_attempt_nbr_outreached')>=2) & 
                                                (F.col('o_channel_seq').like('SMS,%')), 'SMS')\
                                           .otherwise(F.col('o_channel_seq')))\
            .withColumn('last_channel', F.when((F.col('o_rr_attempt_nbr_outreached')>=2) & 
                                                (F.col('o_channel_seq').like('%, IVR')), 'IVR')\
                                           .when((F.col('o_rr_attempt_nbr_outreached')>=2) & 
                                                 (F.col('o_channel_seq').like('%, SMS')), 'SMS')\
                                           .otherwise(F.col('o_channel_seq')))

df_final_clean = df_final.filter(F.col('less_than_3_attempt_flg') == 1)\
                         .filter(F.col('o_attempt_seq').isin(['RR','ARDD','POST2_ARDD',
                                                             'RR, ARDD', 'RR, POST2_ARDD', 'ARDD, POST2_ARDD',
                                                             'RR, ARDD, POST2_ARDD']))\
                         .withColumn('switch', F.when(F.col('first_response') == F.col('last_response'), 0).otherwise(1))\
                         .withColumn('switch_channel', F.when(F.col('first_channel') == F.col('last_channel'), 0).otherwise(1))

In [11]:
%python
import scipy
import pkg_resources
pkg_resources.get_distribution('scipy').version
dbutils.library.list()




Out[ 1 ]: [u'PyPI:scipy-1.2.0-(empty)']

In [12]:
dbutils.library.installPyPI('scipy','1.2.0')
dbutils.library.restartPython()

PyPI package scipy has been installed already. The previous installed scipy has version: 1.2.0 and repo: (empty). To resolve this issue detach and retach the notebook to create a new environment or rename the package.

In [13]:
dbutils.library.installPyPI('quinn')
dbutils.library.restartPython()

In [14]:
%python
import scipy
import pkg_resources
pkg_resources.get_distribution('quinn').version
dbutils.library.list()

Out[ 1 ]: [u'PyPI:quinn-(empty)-(empty)', u'PyPI:scipy-1.2.0-(empty)']

In [15]:
from quinn.extensions import *

In [16]:
print df_final.count(), 
print df_final.select('o_store_nbr', 'o_rx_nbr', 'o_rx_fill_nbr').distinct().count(), 
print df_final.select('store_nbr', 'rx_nbr', 'rx_fill_nbr').distinct().count()

83857243 83857243 46857184

In [17]:
print df_final_clean.count()

80819563

In [18]:
print rxoppty_target.count()

183925263

In [19]:
rxoppty_target.printSchema()

root
-- rx_rxc_pat_id: long (nullable = true)
-- store_nbr: integer (nullable = true)
-- rx_nbr: integer (nullable = true)
-- rx_fill_nbr: short (nullable = true)
-- run_dt: timestamp (nullable = true)
-- pgm_cd: string (nullable = true)
-- sub_pgm_cd: string (nullable = true)
-- con_track_typ: string (nullable = true)
-- dispnd_ndc: string (nullable = true)
-- antcptd_refill_due_dt: timestamp (nullable = true)
-- fill_dt: timestamp (nullable = true)

In [20]:
rxoppty_target.groupby('sub_pgm_cd').count().show()

+----------+--------+
sub_pgm_cd| count|
+----------+--------+
 RR|82187979|
 ARDD|63498223|
POST2_ARDD|38239061|
+----------+--------+

In [21]:
print rx_disp.count()

158322330

In [22]:
rx_disp.groupby('disp_flg').count().show()

+----------+---------+
 disp_flg| count|
+----------+---------+
Outreached|129609305|
 Bypassed| 28713025|
+----------+---------+

In [23]:
rx_disp_a = rx_disp.groupby('disp_flg').count()

In [24]:
rx_disp.filter(F.col('disp_flg')=='Outreached')\
       .groupby('sub_pgm_cd').count()\
       .sort(desc('count')).show()

+----------+--------+
sub_pgm_cd| count|
+----------+--------+
 RR|62164979|
 ARDD|41406278|
POST2_ARDD|26038048|
+----------+--------+

In [25]:
print rx_disp.select('store_nbr', 'rx_nbr', 'rx_fill_nbr').distinct().count()

99984472

In [26]:
print df_final.select('store_nbr', 'rx_nbr', 'rx_fill_nbr').distinct().count()

46857184

In [27]:
print df_final.count()

83857243

In [28]:
df_final.printSchema()

root
-- o_store_nbr: decimal(6,0) (nullable = true)
-- o_rx_nbr: decimal(8,0) (nullable = true)
-- o_rx_fill_nbr: decimal(3,0) (nullable = true)
-- o_run_dt: timestamp (nullable = true)
-- o_sub_pgm_cd: string (nullable = true)
-- o_ivr_call_disp_cd: string (nullable = true)
-- o_disp_flg: string (nullable = true)
-- o_ndc: string (nullable = true)
-- o_ivr_file_typ_cd: string (nullable = true)
-- o_ivr_call_disp_dsc: string (nullable = true)
-- o_ivr_gnrl_disp_cd: string (nullable = true)
-- o_channel: string (nullable = true)
-- o_outreach_typ_cd: string (nullable = true)
-- o_mobile_number: string (nullable = true)
-- o_ivr_refl_req_cd: string (nullable = true)
-- o_ptnt_response_seq: string (nullable = true)
-- o_attempt_seq: string (nullable = true)
-- o_channel_seq: string (nullable = true)
-- o_rx_unique: string (nullable = true)
-- o_num_rx: integer (nullable = true)
-- o_multi_rx_flg: integer (nullable = true)
-- o_pgm_cd: string (nullable = true)
-- o_con_track_typ: string (nullable = true)
-- o_antcptd_refill_due_dt: timestamp (nullable = true)
-- o_fill_dt: timestamp (nullable = true)
-- o_rxc_pat_id: decimal(20,0) (nullable = true)
-- o_gcn: decimal(5,0) (nullable = true)
-- o_gpi4_drug_class: string (nullable = true)
-- o_rr_max_run_dt_outreached: timestamp (nullable = true)
-- o_rr_first_run_dt_outreached: timestamp (nullable = true)
-- o_rr_first_sub_pgm_cd_outreached: string (nullable = true)
-- o_rr_attempt_nbr_outreached: decimal(1,0) (nullable = true)
-- o_rr_first_run_dt: timestamp (nullable = true)
-- o_rr_max_attempt_nbr: decimal(1,0) (nullable = true)
-- o_rr_max_run_dt: timestamp (nullable = true)
-- o_rowNum: integer (nullable = true)
-- o_rowNum2: integer (nullable = true)
-- ndc: string (nullable = true)
-- rxc_pat_id: decimal(18,0) (nullable = true)
-- store_nbr: integer (nullable = true)
-- rx_nbr: decimal(18,0) (nullable = true)
-- rx_fill_nbr: integer (nullable = true)
-- pkup_dt: timestamp (nullable = true)
-- fill_dt: date (nullable = true)
-- gpi4_drug_class: string (nullable = true)
-- rr_time_to_fill_after_reached: integer (nullable = true)
-- rr_time_to_pickup_after_fill: integer (nullable = true)
-- rr_time_to_pickup_after_reached: integer (nullable = true)
-- rr_time_to_pickup_rel_ardd: integer (nullable = true)
-- rr_time_to_fill_rel_ardd: integer (nullable = true)
-- rr_target: integer (nullable = true)
-- n_records_for_target_idenf: long (nullable = true)
-- less_than_3_attempt_flg: integer (nullable = false)
-- valid_fill_flg: integer (nullable = false)
-- first_response: string (nullable = true)
-- last_response: string (nullable = true)
-- first_pgm: string (nullable = true)
-- last_pgm: string (nullable = true)
-- first_channel: string (nullable = true)
-- last_channel: string (nullable = true)

In [29]:
df_final.filter(F.col('o_rr_attempt_nbr_outreached')==3).select('o_rr_attempt_nbr_outreached','o_ptnt_response_seq').show(2)

+---------------------------+-------------------+
o_rr_attempt_nbr_outreached|o_ptnt_response_seq|
+---------------------------+-------------------+
 3| , Y, Y|
 3| , , |
+---------------------------+-------------------+
only showing top 2 rows

In [30]:
df_final.filter(F.col('o_rr_attempt_nbr_outreached')==3).select('o_rr_attempt_nbr_outreached','o_ptnt_response_seq').show(10)

+---------------------------+-------------------+
o_rr_attempt_nbr_outreached|o_ptnt_response_seq|
+---------------------------+-------------------+
 3| , Y, Y|
 3| , , |
 3| , , |
 3| , , |
 3| , , |
 3| , , |
 3| , , |
 3| , , |
 3| , , |
 3| , , |
+---------------------------+-------------------+
only showing top 10 rows

In [31]:
rx_disp.printSchema()

root
-- disp_rxc_pat_id: decimal(18,0) (nullable = true)
-- store_nbr: decimal(6,0) (nullable = true)
-- rx_nbr: decimal(8,0) (nullable = true)
-- rx_fill_nbr: decimal(3,0) (nullable = true)
-- run_dt: timestamp (nullable = true)
-- ivr_file_typ_cd: string (nullable = false)
-- ivr_call_disp_cd: string (nullable = false)
-- ivr_call_disp_dsc: string (nullable = false)
-- ivr_gnrl_disp_cd: string (nullable = false)
-- sub_pgm_cd: string (nullable = false)
-- channel: string (nullable = false)
-- disp_flg: string (nullable = false)
-- outreach_typ_cd: string (nullable = false)
-- mobile_number: string (nullable = false)
-- ivr_refl_req_cd: string (nullable = false)
-- ptnt_response_seq: string (nullable = false)
-- attempt_seq: string (nullable = false)
-- channel_seq: string (nullable = false)
-- rx_unique: string (nullable = true)
-- num_rx: integer (nullable = false)
-- multi_rx_flg: integer (nullable = false)

In [32]:
rx_disp.groupby('ptnt_response_seq').count()\
       .sort(desc('count')).show()

+-----------------+--------+
ptnt_response_seq| count|
+-----------------+--------+
 NULL|40281716|
 Y|21823572|
 NULL, NULL|16957632|
 |14432486|
 S|11448275|
 N| 9873616|
 , | 5330413|
 NULL, NULL, NULL| 5116671|
 N, N| 4096760|
 Y, Y| 3019886|
 NULL, Y| 2426280|
 S, S| 1919912|
 NULL, N| 1624669|
 NULL, S| 1607484|
 Y, S| 1381194|
 N, NULL| 1349700|
 N, N, N| 1163609|
 NULL, | 954546|
 , , | 923885|
 , Y| 666407|
+-----------------+--------+
only showing top 20 rows

In [33]:
rx_disp.filter((F.col('ptnt_response_seq') == 'Y, Y')).select('store_nbr', 'rx_nbr', 'rx_fill_nbr','run_dt').show()


+---------+-------+-----------+-------------------+
store_nbr| rx_nbr|rx_fill_nbr| run_dt|
+---------+-------+-----------+-------------------+
 3| 702909| 0|2018-08-27 00:00:00|
 6| 575072| 0|2018-10-26 00:00:00|
 8|2109345| 3|2018-08-19 00:00:00|
 8|2109345| 3|2018-08-19 00:00:00|
 8|2154191| 0|2018-07-15 00:00:00|
 8|2154191| 0|2018-07-15 00:00:00|
 8|2173503| 1|2019-01-22 00:00:00|
 8|2173503| 1|2019-01-22 00:00:00|
 8|2187616| 0|2018-12-01 00:00:00|
 8|2187616| 0|2018-12-01 00:00:00|
 9|1393514| 0|2018-07-28 00:00:00|
 9|1393514| 0|2018-07-28 00:00:00|
 11| 536334| 0|2018-07-01 00:00:00|
 11| 568463| 0|2018-10-26 00:00:00|
 11| 568463| 0|2018-10-26 00:00:00|
 12|1317951| 3|2018-07-21 00:00:00|
 12|1381152| 1|2018-09-07 00:00:00|
 12|1381152| 1|2018-09-07 00:00:00|
 17|1456893| 6|2018-07-07 00:00:00|
 17|1456893| 6|2018-07-07 00:00:00|
+---------+-------+-----------+-------------------+
only showing top 20 rows

In [34]:
rx_disp.filter(F.col('store_nbr') == 8) \
        .filter(F.col('rx_nbr') == '2154191').show(1000)

+---------------+---------+-------+-----------+-------------------+---------------+----------------+--------------------+----------------+----------+-------+----------+---------------+-------------+---------------+-----------------+-----------+-----------+---------+------+------------+
disp_rxc_pat_id|store_nbr| rx_nbr|rx_fill_nbr| run_dt|ivr_file_typ_cd|ivr_call_disp_cd| ivr_call_disp_dsc|ivr_gnrl_disp_cd|sub_pgm_cd|channel| disp_flg|outreach_typ_cd|mobile_number|ivr_refl_req_cd|ptnt_response_seq|attempt_seq|channel_seq|rx_unique|num_rx|multi_rx_flg|
+---------------+---------+-------+-----------+-------------------+---------------+----------------+--------------------+----------------+----------+-------+----------+---------------+-------------+---------------+-----------------+-----------+-----------+---------+------+------------+
 4995685061| 8|2154191| 0|2018-07-15 00:00:00| RR| SMS-E2|Received Help Mes...| SMS| RR| SMS|Outreached| RR|0014692798226| Y| Y, Y| RR, RR| SMS, SMS|821541910| 2| 1|
 4995685061| 8|2154191| 0|2018-07-15 00:00:00| RR| SMS-F2|Received Multi Rx...| SMS| RR| SMS|Outreached| RR|0014692798226| Y| Y, Y| RR, RR| SMS, SMS|821541910| 2| 1|
 4995685061| 8|2154191| 1|2018-08-14 00:00:00| RR| SMS| NULL| NULL| RR| SMS|Outreached| RR|0014692798226| NULL| NULL| RR| SMS|821541911| 6| 1|
 4995685061| 8|2154191| 1|2018-08-20 00:00:00| RR| SMS| NULL| NULL| ARDD| SMS|Outreached| ARDD|0014692798226| NULL| NULL, NULL| RR, ARDD| SMS, SMS|821541911| 6| 1|
+---------------+---------+-------+-----------+-------------------+---------------+----------------+--------------------+----------------+----------+-------+----------+---------------+-------------+---------------+-----------------+-----------+-----------+---------+------+------------+

In [35]:
print rx_disp.filter(F.col('store_nbr') == 8)\
       .filter(F.col('rx_nbr') == '2154191')\
       .filter(F.col('rx_fill_nbr') == 0 ).select('store_nbr','rx_nbr','rx_fill_nbr','run_dt','ivr_refl_req_cd','ptnt_response_seq').show()

+---------+-------+-----------+-------------------+---------------+-----------------+
store_nbr| rx_nbr|rx_fill_nbr| run_dt|ivr_refl_req_cd|ptnt_response_seq|
+---------+-------+-----------+-------------------+---------------+-----------------+
 8|2154191| 0|2018-07-15 00:00:00| Y| Y, Y|
 8|2154191| 0|2018-07-15 00:00:00| Y| Y, Y|
+---------+-------+-----------+-------------------+---------------+-----------------+

None

In [36]:
print rx_disp.filter(F.col('store_nbr') == 8)\
       .filter(F.col('rx_nbr') == '2154191')\
       .filter(F.col('rx_fill_nbr') == 1 ).select('store_nbr','rx_nbr','rx_fill_nbr','run_dt','ivr_refl_req_cd','ptnt_response_seq').show()

+---------+-------+-----------+-------------------+---------------+-----------------+
store_nbr| rx_nbr|rx_fill_nbr| run_dt|ivr_refl_req_cd|ptnt_response_seq|
+---------+-------+-----------+-------------------+---------------+-----------------+
 8|2154191| 1|2018-08-14 00:00:00| NULL| NULL|
 8|2154191| 1|2018-08-20 00:00:00| NULL| NULL, NULL|
+---------+-------+-----------+-------------------+---------------+-----------------+

None

In [37]:
rx_disp.filter((F.col('ptnt_response_seq') == 'N, N, N')).select('store_nbr', 'rx_nbr', 'rx_fill_nbr','run_dt').show(100)

+---------+-------+-----------+-------------------+
store_nbr| rx_nbr|rx_fill_nbr| run_dt|
+---------+-------+-----------+-------------------+
 3| 722470| 0|2018-12-24 00:00:00|
 3| 722470| 0|2018-12-24 00:00:00|
 6| 568031| 0|2018-08-13 00:00:00|
 8|2149482| 2|2018-09-13 00:00:00|
 8|2149482| 2|2018-09-13 00:00:00|
 8|2197174| 2|2018-12-29 00:00:00|
 8|2209530| 0|2019-01-31 00:00:00|
 12|1387813| 0|2018-07-25 00:00:00|
 16| 578949| 0|2019-01-25 00:00:00|
 17|1534826| 2|2018-09-25 00:00:00|
 19| 949211| 0|2018-10-27 00:00:00|
 20|1563517| 3|2018-11-29 00:00:00|
 20|1574459| 0|2018-10-10 00:00:00|
 20|1574459| 0|2018-10-10 00:00:00|
 22|2665219| 1|2018-09-26 00:00:00|
 23|1013846| 2|2018-10-12 00:00:00|
 25| 658506| 0|2018-08-14 00:00:00|
 38| 997133| 2|2018-09-07 00:00:00|
 38|1046560| 0|2018-10-19 00:00:00|
 41|1508375| 1|2019-01-30 00:00:00|
 46| 993182| 4|2018-12-21 00:00:00|
 46|1009592| 1|2018-10-26 00:00:00|
 47| 324996| 4|2018-09-03 00:00:00|
 51| 574336| 1|2018-09-20 00:00:00|
 52| 867317| 0|2018-10-17 00:00:00|
 53|1391653| 0|2018-10-04 00:00:00|
 53|1391653| 0|2018-10-04 00:00:00|
 53|1404870| 0|2018-09-20 00:00:00|
 53|1437010| 0|2019-01-29 00:00:00|
 59| 659604| 0|2018-12-11 00:00:00|
 68| 693473| 1|2019-01-17 00:00:00|
 69|1172696| 2|2019-01-27 00:00:00|
 70| 581812| 0|2019-01-14 00:00:00|
 73|1688098| 1|2018-08-09 00:00:00|
 74|1202160| 3|2018-09-14 00:00:00|
 74|1212912| 2|2018-07-23 00:00:00|
 78| 929328| 3|2018-08-23 00:00:00|
 79| 617270| 0|2018-07-26 00:00:00|
 80| 824416| 0|2018-10-28 00:00:00|
 84| 349832| 3|2018-09-28 00:00:00|
 85| 424708| 0|2018-11-09 00:00:00|
 103| 778400| 0|2018-09-04 00:00:00|
 104| 691772| 7|2018-09-02 00:00:00|
 113| 760981| 0|2018-07-15 00:00:00|
 114|1981435| 0|2018-08-28 00:00:00|
 114|2006872| 0|2018-10-28 00:00:00|
 114|2006872| 0|2018-10-28 00:00:00|
 114|2029698| 0|2019-01-07 00:00:00|
 116| 582518| 0|2018-08-14 00:00:00|
 121| 552597| 0|2018-07-29 00:00:00|
 123|1552010| 0|2018-08-16 00:00:00|
 123|1570908| 0|2018-10-15 00:00:00|
 125|1098838| 0|2018-10-09 00:00:00|
 131|1261109| 2|2019-01-12 00:00:00|
 131|1261109| 2|2019-01-12 00:00:00|
 131|1294242| 0|2018-09-13 00:00:00|
 131|1314410| 2|2019-01-28 00:00:00|
 131|1314410| 2|2019-01-28 00:00:00|
 137|2039338| 0|2018-10-29 00:00:00|
 140| 903287| 0|2018-08-05 00:00:00|
 141| 645179| 1|2018-09-09 00:00:00|
 141| 658408| 0|2018-10-18 00:00:00|
 142| 796299| 0|2018-09-03 00:00:00|
 142| 806293| 0|2018-11-06 00:00:00|
 143| 515887| 0|2018-12-28 00:00:00|
 148| 738646| 0|2018-08-07 00:00:00|
 149|1072335| 0|2018-07-07 00:00:00|
 149|1072335| 0|2018-07-07 00:00:00|
 149|1079612| 1|2018-07-10 00:00:00|
 149|1079612| 1|2018-07-10 00:00:00|
 154| 965971| 0|2018-09-02 00:00:00|
 162| 476947| 0|2018-07-09 00:00:00|
 163|1039340| 3|2018-07-16 00:00:00|
 163|1103884| 0|2019-01-31 00:00:00|
 164| 409630| 0|2018-06-12 00:00:00|
 164| 415508| 0|2018-07-27 00:00:00|
 164| 415508| 0|2018-07-27 00:00:00|
 166|2077096| 0|2018-09-20 00:00:00|
 167| 412793| 1|2018-08-19 00:00:00|
 167| 413894| 0|2018-07-16 00:00:00|
 168| 622321| 0|2018-07-25 00:00:00|
 168| 657849| 0|2019-01-28 00:00:00|
 169| 711274| 0|2018-07-10 00:00:00|
 172|1081654| 0|2018-08-16 00:00:00|
 174| 633921| 1|2018-07-18 00:00:00|
 176| 781898| 0|2018-10-08 00:00:00|
 181|1490624| 1|2018-07-07 00:00:00|
 181|1490624| 1|2018-07-07 00:00:00|
 181|1540443| 0|2019-01-06 00:00:00|
 182| 324228| 0|2018-09-19 00:00:00|
 183|1345772| 3|2018-07-06 00:00:00|
 188|1232555| 0|2018-06-29 00:00:00|
 203|1221906| 0|2018-09-08 00:00:00|
 204|1066090| 0|2018-11-03 00:00:00|
 205| 507488| 1|2018-11-21 00:00:00|
 208| 606434| 0|2018-08-16 00:00:00|
 208| 612109| 1|2018-10-23 00:00:00|
 209|1061405| 0|2018-07-01 00:00:00|
 210|1220250| 3|2019-01-22 00:00:00|
 210|1239229| 0|2018-10-06 00:00:00|
+---------+-------+-----------+-------------------+
only showing top 100 rows

In [38]:
print rx_disp.filter(F.col('store_nbr') == 3)\
       .filter(F.col('rx_nbr') == '722470')\
       .filter(F.col('rx_fill_nbr') == 0 ).select('store_nbr','rx_nbr','rx_fill_nbr','run_dt','ivr_refl_req_cd','ptnt_response_seq').show()

+---------+------+-----------+-------------------+---------------+-----------------+
store_nbr|rx_nbr|rx_fill_nbr| run_dt|ivr_refl_req_cd|ptnt_response_seq|
+---------+------+-----------+-------------------+---------------+-----------------+
 3|722470| 0|2018-12-07 00:00:00| N| N|
 3|722470| 0|2018-12-24 00:00:00| N| N, N, N|
 3|722470| 0|2018-12-24 00:00:00| N| N, N, N|
+---------+------+-----------+-------------------+---------------+-----------------+

None

In [39]:
print rx_disp.filter(F.col('store_nbr') == 47)\
       .filter(F.col('rx_nbr') == '324996')\
       .filter(F.col('rx_fill_nbr') == 4 ).select('store_nbr','rx_nbr','rx_fill_nbr','run_dt','ivr_refl_req_cd','ptnt_response_seq').show()

+---------+------+-----------+-------------------+---------------+-----------------+
store_nbr|rx_nbr|rx_fill_nbr| run_dt|ivr_refl_req_cd|ptnt_response_seq|
+---------+------+-----------+-------------------+---------------+-----------------+
 47|324996| 4|2018-08-12 00:00:00| N| N|
 47|324996| 4|2018-08-18 00:00:00| N| N, N|
 47|324996| 4|2018-09-03 00:00:00| N| N, N, N|
+---------+------+-----------+-------------------+---------------+-----------------+

None

In [40]:
df_final.printSchema()

root
-- o_store_nbr: decimal(6,0) (nullable = true)
-- o_rx_nbr: decimal(8,0) (nullable = true)
-- o_rx_fill_nbr: decimal(3,0) (nullable = true)
-- o_run_dt: timestamp (nullable = true)
-- o_sub_pgm_cd: string (nullable = true)
-- o_ivr_call_disp_cd: string (nullable = true)
-- o_disp_flg: string (nullable = true)
-- o_ndc: string (nullable = true)
-- o_ivr_file_typ_cd: string (nullable = true)
-- o_ivr_call_disp_dsc: string (nullable = true)
-- o_ivr_gnrl_disp_cd: string (nullable = true)
-- o_channel: string (nullable = true)
-- o_outreach_typ_cd: string (nullable = true)
-- o_mobile_number: string (nullable = true)
-- o_ivr_refl_req_cd: string (nullable = true)
-- o_ptnt_response_seq: string (nullable = true)
-- o_attempt_seq: string (nullable = true)
-- o_channel_seq: string (nullable = true)
-- o_rx_unique: string (nullable = true)
-- o_num_rx: integer (nullable = true)
-- o_multi_rx_flg: integer (nullable = true)
-- o_pgm_cd: string (nullable = true)
-- o_con_track_typ: string (nullable = true)
-- o_antcptd_refill_due_dt: timestamp (nullable = true)
-- o_fill_dt: timestamp (nullable = true)
-- o_rxc_pat_id: decimal(20,0) (nullable = true)
-- o_gcn: decimal(5,0) (nullable = true)
-- o_gpi4_drug_class: string (nullable = true)
-- o_rr_max_run_dt_outreached: timestamp (nullable = true)
-- o_rr_first_run_dt_outreached: timestamp (nullable = true)
-- o_rr_first_sub_pgm_cd_outreached: string (nullable = true)
-- o_rr_attempt_nbr_outreached: decimal(1,0) (nullable = true)
-- o_rr_first_run_dt: timestamp (nullable = true)
-- o_rr_max_attempt_nbr: decimal(1,0) (nullable = true)
-- o_rr_max_run_dt: timestamp (nullable = true)
-- o_rowNum: integer (nullable = true)
-- o_rowNum2: integer (nullable = true)
-- ndc: string (nullable = true)
-- rxc_pat_id: decimal(18,0) (nullable = true)
-- store_nbr: integer (nullable = true)
-- rx_nbr: decimal(18,0) (nullable = true)
-- rx_fill_nbr: integer (nullable = true)
-- pkup_dt: timestamp (nullable = true)
-- fill_dt: date (nullable = true)
-- gpi4_drug_class: string (nullable = true)
-- rr_time_to_fill_after_reached: integer (nullable = true)
-- rr_time_to_pickup_after_fill: integer (nullable = true)
-- rr_time_to_pickup_after_reached: integer (nullable = true)
-- rr_time_to_pickup_rel_ardd: integer (nullable = true)
-- rr_time_to_fill_rel_ardd: integer (nullable = true)
-- rr_target: integer (nullable = true)
-- n_records_for_target_idenf: long (nullable = true)
-- less_than_3_attempt_flg: integer (nullable = false)
-- valid_fill_flg: integer (nullable = false)
-- first_response: string (nullable = true)
-- last_response: string (nullable = true)
-- first_pgm: string (nullable = true)
-- last_pgm: string (nullable = true)
-- first_channel: string (nullable = true)
-- last_channel: string (nullable = true)

In [41]:


print df_final.filter(F.col('o_store_nbr') == 47)\
       .filter(F.col('o_rx_nbr') == '324996')\
       .filter(F.col('o_rx_fill_nbr') == 4 ).select('o_store_nbr','o_rx_nbr','o_rx_fill_nbr','o_rr_max_run_dt','o_ivr_refl_req_cd','o_ptnt_response_seq').show()

+-----------+--------+-------------+-------------------+-----------------+-------------------+
o_store_nbr|o_rx_nbr|o_rx_fill_nbr| o_rr_max_run_dt|o_ivr_refl_req_cd|o_ptnt_response_seq|
+-----------+--------+-------------+-------------------+-----------------+-------------------+
 47| 324996| 4|2018-09-03 00:00:00| N| N, N, N|
+-----------+--------+-------------+-------------------+-----------------+-------------------+

None

In [42]:
print df_final.count()

83857243

In [43]:
df_final\
    .groupby('o_ptnt_response_seq')\
    .agg((F.count('*')).alias('count'), ((F.count('*')/df_final.count()).cast("decimal(18,2)")).alias('percentage'))\
    .sort(desc('o_ptnt_response_seq')).show(1000)

+--------------------+--------+----------+
 o_ptnt_response_seq| count|percentage|
+--------------------+--------+----------+
 Y, Y, Y, Y, Y, Y, Y| 1| 0.00|
Y, Y, Y, Y, Y, Y, , | 1| 0.00|
 Y, Y, Y, Y, Y, Y| 20| 0.00|
 Y, Y, Y, Y, Y, NULL| 2| 0.00|
 Y, Y, Y, Y, Y| 207| 0.00|
 Y, Y, Y, Y, S, Y| 1| 0.00|
 Y, Y, Y, Y, S, S| 1| 0.00|
 Y, Y, Y, Y, S, NULL| 2| 0.00|
 Y, Y, Y, Y, S| 2| 0.00|
 Y, Y, Y, Y, NULL, Y| 1| 0.00|
 Y, Y, Y, Y, NULL| 57| 0.00|
 Y, Y, Y, Y, N, N, N| 1| 0.00|
 Y, Y, Y, Y, N, N| 3| 0.00|
 Y, Y, Y, Y, N| 16| 0.00|
 Y, Y, Y, Y, | 14| 0.00|
 Y, Y, Y, Y| 4515| 0.00|
 Y, Y, Y, S, Y, Y, | 1| 0.00|
 Y, Y, Y, S, Y, Y| 7| 0.00|
 Y, Y, Y, S, Y| 41| 0.00|
 Y, Y, Y, S, S, NULL| 1| 0.00|
 Y, Y, Y, S, S| 8| 0.00|
 Y, Y, Y, S, NULL| 52| 0.00|
 Y, Y, Y, S, N| 13| 0.00|
 Y, Y, Y, S, | 4| 0.00|
 Y, Y, Y, S| 147| 0.00|
 Y, Y, Y, NULL, Y, Y| 1| 0.00|
 Y, Y, Y, NULL, Y| 12| 0.00|
 Y, Y, Y, NULL, S| 1| 0.00|
Y, Y, Y, NULL, NU...| 1| 0.00|
Y, Y, Y, NULL, NU...| 1| 0.00|
Y, Y, Y, NULL, NU...| 1| 0.00|
Y, Y, Y, NULL, NU...| 1| 0.00|
 Y, Y, Y, NULL, NULL| 34| 0.00|
 Y, Y, Y, NULL, N| 5| 0.00|
 Y, Y, Y, NULL, , | 1| 0.00|
 Y, Y, Y, NULL, | 4| 0.00|
 Y, Y, Y, NULL| 1115| 0.00|
 Y, Y, Y, N, NULL| 4| 0.00|
 Y, Y, Y, N, N| 36| 0.00|
 Y, Y, Y, N, | 1| 0.00|
 Y, Y, Y, N| 312| 0.00|
 Y, Y, Y, , Y| 3| 0.00|
 Y, Y, Y, , NULL| 1| 0.00|
 Y, Y, Y, , | 1| 0.00|
 Y, Y, Y, | 226| 0.00|
 Y, Y, Y| 71247| 0.00|
 Y, Y, S, Y, Y, Y| 3| 0.00|
 Y, Y, S, Y, Y, NULL| 4| 0.00|
 Y, Y, S, Y, Y| 685| 0.00|
 Y, Y, S, Y, NULL, Y| 1| 0.00|
 Y, Y, S, Y, NULL| 3| 0.00|
 Y, Y, S, Y| 5872| 0.00|
 Y, Y, S, S, Y, Y| 7| 0.00|
 Y, Y, S, S, Y| 33| 0.00|
 Y, Y, S, S, S, Y, Y| 1| 0.00|
 Y, Y, S, S, S| 34| 0.00|
 Y, Y, S, S, NULL| 31| 0.00|
 Y, Y, S, S, N, N| 1| 0.00|
 Y, Y, S, S, N| 14| 0.00|
 Y, Y, S, S, | 10| 0.00|
 Y, Y, S, S| 1345| 0.00|
 Y, Y, S, NULL, Y| 3| 0.00|
 Y, Y, S, NULL, NULL| 45| 0.00|
 Y, Y, S, NULL, | 40| 0.00|
 Y, Y, S, NULL| 7462| 0.00|
 Y, Y, S, N, NULL, N| 1| 0.00|
 Y, Y, S, N, NULL| 2| 0.00|
 Y, Y, S, N, N| 194| 0.00|
 Y, Y, S, N| 2815| 0.00|
 Y, Y, S, , NULL| 33| 0.00|
 Y, Y, S, , | 5| 0.00|
 Y, Y, S, | 3163| 0.00|
 Y, Y, S| 15240| 0.00|
Y, Y, NULL, Y, Y,...| 1| 0.00|
 Y, Y, NULL, Y, Y| 71| 0.00|
 Y, Y, NULL, Y, S, Y| 1| 0.00|
 Y, Y, NULL, Y, NULL| 7| 0.00|
 Y, Y, NULL, Y, N| 1| 0.00|
 Y, Y, NULL, Y, | 1| 0.00|
 Y, Y, NULL, Y| 470| 0.00|
 Y, Y, NULL, S, Y| 4| 0.00|
 Y, Y, NULL, S, S| 2| 0.00|
 Y, Y, NULL, S, NULL| 5| 0.00|
 Y, Y, NULL, S, N| 1| 0.00|
 Y, Y, NULL, S, | 1| 0.00|
 Y, Y, NULL, S| 27| 0.00|
Y, Y, NULL, NULL,...| 4| 0.00|
 Y, Y, NULL, NULL, Y| 7| 0.00|
Y, Y, NULL, NULL,...| 1| 0.00|
Y, Y, NULL, NULL,...| 32| 0.00|
Y, Y, NULL, NULL,...| 3| 0.00|
 Y, Y, NULL, NULL, N| 4| 0.00|
Y, Y, NULL, NULL, , | 1| 0.00|
 Y, Y, NULL, NULL, | 10| 0.00|
 Y, Y, NULL, NULL| 1614| 0.00|
 Y, Y, NULL, N, NULL| 2| 0.00|
 Y, Y, NULL, N, N, N| 1| 0.00|
 Y, Y, NULL, N, N| 25| 0.00|
 Y, Y, NULL, N| 295| 0.00|
 Y, Y, NULL, , Y| 4| 0.00|
 Y, Y, NULL, , NULL| 4| 0.00|
 Y, Y, NULL, , N| 1| 0.00|
 Y, Y, NULL, , | 2| 0.00|
 Y, Y, NULL, | 183| 0.00|
 Y, Y, NULL| 41416| 0.00|
 Y, Y, N, Y, Y| 8| 0.00|
 Y, Y, N, Y| 54| 0.00|
 Y, Y, N, S, NULL| 1| 0.00|
 Y, Y, N, S| 8| 0.00|
 Y, Y, N, NULL, Y| 1| 0.00|
 Y, Y, N, NULL, NULL| 4| 0.00|
 Y, Y, N, NULL, N| 1| 0.00|
 Y, Y, N, NULL| 211| 0.00|
 Y, Y, N, N, Y, Y| 2| 0.00|
 Y, Y, N, N, Y| 8| 0.00|
 Y, Y, N, N, NULL| 22| 0.00|
 Y, Y, N, N, N, NULL| 1| 0.00|
 Y, Y, N, N, N, N, N| 1| 0.00|
 Y, Y, N, N, N, N| 7| 0.00|
 Y, Y, N, N, N| 61| 0.00|
 Y, Y, N, N, | 3| 0.00|
 Y, Y, N, N| 1343| 0.00|
 Y, Y, N, | 44| 0.00|
 Y, Y, N| 12802| 0.00|
 Y, Y, , Y, Y| 20| 0.00|
 Y, Y, , Y| 82| 0.00|
 Y, Y, , S, Y| 1| 0.00|
 Y, Y, , S, | 2| 0.00|
 Y, Y, , S| 2| 0.00|
 Y, Y, , NULL, Y, Y| 1| 0.00|
 Y, Y, , NULL, N| 2| 0.00|
 Y, Y, , NULL| 105| 0.00|
 Y, Y, , N, N, | 1| 0.00|
 Y, Y, , N, N| 12| 0.00|
 Y, Y, , N, | 1| 0.00|
 Y, Y, , N| 46| 0.00|
 Y, Y, , , Y| 1| 0.00|
 Y, Y, , , N, N| 1| 0.00|
 Y, Y, , , N| 1| 0.00|
 Y, Y, , , | 1| 0.00|
 Y, Y, , | 238| 0.00|
 Y, Y, | 115

In [44]:
df_final\
    .groupby('o_ptnt_response_seq').count()\
    .sort(desc('o_ptnt_response_seq')).show(1000)

+--------------------+--------+
 o_ptnt_response_seq| count|
+--------------------+--------+
 Y, Y, Y, Y, Y, Y, Y| 1|
Y, Y, Y, Y, Y, Y, , | 1|
 Y, Y, Y, Y, Y, Y| 20|
 Y, Y, Y, Y, Y, NULL| 2|
 Y, Y, Y, Y, Y| 207|
 Y, Y, Y, Y, S, Y| 1|
 Y, Y, Y, Y, S, S| 1|
 Y, Y, Y, Y, S, NULL| 2|
 Y, Y, Y, Y, S| 2|
 Y, Y, Y, Y, NULL, Y| 1|
 Y, Y, Y, Y, NULL| 57|
 Y, Y, Y, Y, N, N, N| 1|
 Y, Y, Y, Y, N, N| 3|
 Y, Y, Y, Y, N| 16|
 Y, Y, Y, Y, | 14|
 Y, Y, Y, Y| 4515|
 Y, Y, Y, S, Y, Y, | 1|
 Y, Y, Y, S, Y, Y| 7|
 Y, Y, Y, S, Y| 41|
 Y, Y, Y, S, S, NULL| 1|
 Y, Y, Y, S, S| 8|
 Y, Y, Y, S, NULL| 52|
 Y, Y, Y, S, N| 13|
 Y, Y, Y, S, | 4|
 Y, Y, Y, S| 147|
 Y, Y, Y, NULL, Y, Y| 1|
 Y, Y, Y, NULL, Y| 12|
 Y, Y, Y, NULL, S| 1|
Y, Y, Y, NULL, NU...| 1|
Y, Y, Y, NULL, NU...| 1|
Y, Y, Y, NULL, NU...| 1|
Y, Y, Y, NULL, NU...| 1|
 Y, Y, Y, NULL, NULL| 34|
 Y, Y, Y, NULL, N| 5|
 Y, Y, Y, NULL, , | 1|
 Y, Y, Y, NULL, | 4|
 Y, Y, Y, NULL| 1115|
 Y, Y, Y, N, NULL| 4|
 Y, Y, Y, N, N| 36|
 Y, Y, Y, N, | 1|
 Y, Y, Y, N| 312|
 Y, Y, Y, , Y| 3|
 Y, Y, Y, , NULL| 1|
 Y, Y, Y, , | 1|
 Y, Y, Y, | 226|
 Y, Y, Y| 71247|
 Y, Y, S, Y, Y, Y| 3|
 Y, Y, S, Y, Y, NULL| 4|
 Y, Y, S, Y, Y| 685|
 Y, Y, S, Y, NULL, Y| 1|
 Y, Y, S, Y, NULL| 3|
 Y, Y, S, Y| 5872|
 Y, Y, S, S, Y, Y| 7|
 Y, Y, S, S, Y| 33|
 Y, Y, S, S, S, Y, Y| 1|
 Y, Y, S, S, S| 34|
 Y, Y, S, S, NULL| 31|
 Y, Y, S, S, N, N| 1|
 Y, Y, S, S, N| 14|
 Y, Y, S, S, | 10|
 Y, Y, S, S| 1345|
 Y, Y, S, NULL, Y| 3|
 Y, Y, S, NULL, NULL| 45|
 Y, Y, S, NULL, | 40|
 Y, Y, S, NULL| 7462|
 Y, Y, S, N, NULL, N| 1|
 Y, Y, S, N, NULL| 2|
 Y, Y, S, N, N| 194|
 Y, Y, S, N| 2815|
 Y, Y, S, , NULL| 33|
 Y, Y, S, , | 5|
 Y, Y, S, | 3163|
 Y, Y, S| 15240|
Y, Y, NULL, Y, Y,...| 1|
 Y, Y, NULL, Y, Y| 71|
 Y, Y, NULL, Y, S, Y| 1|
 Y, Y, NULL, Y, NULL| 7|
 Y, Y, NULL, Y, N| 1|
 Y, Y, NULL, Y, | 1|
 Y, Y, NULL, Y| 470|
 Y, Y, NULL, S, Y| 4|
 Y, Y, NULL, S, S| 2|
 Y, Y, NULL, S, NULL| 5|
 Y, Y, NULL, S, N| 1|
 Y, Y, NULL, S, | 1|
 Y, Y, NULL, S| 27|
Y, Y, NULL, NULL,...| 4|
 Y, Y, NULL, NULL, Y| 7|
Y, Y, NULL, NULL,...| 1|
Y, Y, NULL, NULL,...| 32|
Y, Y, NULL, NULL,...| 3|
 Y, Y, NULL, NULL, N| 4|
Y, Y, NULL, NULL, , | 1|
 Y, Y, NULL, NULL, | 10|
 Y, Y, NULL, NULL| 1614|
 Y, Y, NULL, N, NULL| 2|
 Y, Y, NULL, N, N, N| 1|
 Y, Y, NULL, N, N| 25|
 Y, Y, NULL, N| 295|
 Y, Y, NULL, , Y| 4|
 Y, Y, NULL, , NULL| 4|
 Y, Y, NULL, , N| 1|
 Y, Y, NULL, , | 2|
 Y, Y, NULL, | 183|
 Y, Y, NULL| 41416|
 Y, Y, N, Y, Y| 8|
 Y, Y, N, Y| 54|
 Y, Y, N, S, NULL| 1|
 Y, Y, N, S| 8|
 Y, Y, N, NULL, Y| 1|
 Y, Y, N, NULL, NULL| 4|
 Y, Y, N, NULL, N| 1|
 Y, Y, N, NULL| 211|
 Y, Y, N, N, Y, Y| 2|
 Y, Y, N, N, Y| 8|
 Y, Y, N, N, NULL| 22|
 Y, Y, N, N, N, NULL| 1|
 Y, Y, N, N, N, N, N| 1|
 Y, Y, N, N, N, N| 7|
 Y, Y, N, N, N| 61|
 Y, Y, N, N, | 3|
 Y, Y, N, N| 1343|
 Y, Y, N, | 44|
 Y, Y, N| 12802|
 Y, Y, , Y, Y| 20|
 Y, Y, , Y| 82|
 Y, Y, , S, Y| 1|
 Y, Y, , S, | 2|
 Y, Y, , S| 2|
 Y, Y, , NULL, Y, Y| 1|
 Y, Y, , NULL, N| 2|
 Y, Y, , NULL| 105|
 Y, Y, , N, N, | 1|
 Y, Y, , N, N| 12|
 Y, Y, , N, | 1|
 Y, Y, , N| 46|
 Y, Y, , , Y| 1|
 Y, Y, , , N, N| 1|
 Y, Y, , , N| 1|
 Y, Y, , , | 1|
 Y, Y, , | 238|
 Y, Y, | 11539|
 Y, Y| 1636156|
 Y, S, Y, Y, Y| 11|
Y, S, Y, Y, S, NU...| 1|
 Y, S, Y, Y, NULL| 3|
 Y, S, Y, Y, | 1|
 Y, S, Y, Y| 4012|
 Y, S, Y, S, Y| 2|
 Y, S, Y, NULL, Y, Y| 1|
 Y, S, Y, NULL, Y| 2|
 Y, S, Y, NULL, N| 1|
 Y, S, Y, NULL| 31|
 Y, S, Y, N| 1|
 Y, S, Y, , Y| 2|
 Y, S, Y, | 6|
 Y, S, Y| 98861|
 Y, S, S, Y, Y| 26|
 Y, S, S, Y| 357|
 Y, S, S, S, Y, | 1|
 Y, S, S, S, Y| 3|
 Y, S, S, S, S| 7|
 Y, S, S, S, NULL| 3|
 Y, S, S, S| 237|
 Y, S, S, NULL, NULL| 11|
 Y, S, S, NULL, | 2|
 Y, S, S, NULL| 368|
 Y, S, S, N, N| 4|
 Y, S, S, N| 102|
 Y, S, S, , NULL| 1|
 Y, S, S, | 85|
 Y, S, S| 11145|
 Y, S, NULL, Y, Y| 3|
 Y, S, NULL, Y| 28|
Y, S, NULL, NULL,...| 4|
 Y, S, NULL, NULL, | 1|
 Y, S, NULL, NULL| 310|
 Y, S, NULL, N, NULL| 1|
 Y, S, NULL, N, N| 2|
 Y, S, NULL, N| 8|
 Y, S, NULL, | 297|
 Y, S, NULL| 92551|
 Y, S, N, Y, Y| 1|
 Y, S, N, Y| 2|
 Y, S, N, NULL, NULL| 1|
 Y, 

In [45]:
df_final\
    .filter(F.col('less_than_3_attempt_flg') == 1)\
    .filter(F.col('o_rr_attempt_nbr_outreached') == 1)\
    .groupby('o_ptnt_response_seq','less_than_3_attempt_flg', 'o_rr_attempt_nbr_outreached')\
    .agg(F.count('*').alias('count'), 
         (F.when(F.col('o_rr_attempt_nbr_outreached') == 1, (F.count('*')/48937081).cast("decimal(18,2)"))\
           .when(F.col('o_rr_attempt_nbr_outreached') == 2, (F.count('*')/24571819).cast("decimal(18,2)"))\
           .when(F.col('o_rr_attempt_nbr_outreached') == 3, (F.count('*')/9893367).cast("decimal(18,2)"))).alias('percentage'), 
         (F.avg('valid_fill_flg').cast("decimal(18,2)")).alias('valid_fill_rate'),  
         (F.avg('rr_target').cast("decimal(18,2)")).alias('pickup_rate'))\
    .sort('o_ptnt_response_seq').show(1000)

+--------------------+-----------------------+---------------------------+--------+----------+---------------+-----------+
 o_ptnt_response_seq|less_than_3_attempt_flg|o_rr_attempt_nbr_outreached| count|percentage|valid_fill_rate|pickup_rate|
+--------------------+-----------------------+---------------------------+--------+----------+---------------+-----------+
 | 1| 1| 6602451| 0.13| 0.55| 0.48|
 , | 1| 1| 33718| 0.00| 0.34| 0.29|
 , , | 1| 1| 2529| 0.00| 0.08| 0.06|
 , , N| 1| 1| 484| 0.00| 0.05| 0.03|
 , , S| 1| 1| 42| 0.00| 0.12| 0.10|
 , , Y| 1| 1| 346| 0.00| 0.75| 0.55|
 , N| 1| 1| 4225| 0.00| 0.25| 0.21|
 , N, | 1| 1| 1| 0.00| 0.00| 0.00|
 , NULL| 1| 1| 368| 0.00| 0.57| 0.51|
 , NULL, | 1| 1| 572| 0.00| 0.06| 0.05|
 , NULL, N| 1| 1| 17| 0.00| 0.06| 0.00|
 , NULL, S| 1| 1| 2| 0.00| 0.00| 0.00|
 , NULL, Y| 1| 1| 17| 0.00| 0.82| 0.59|
 , S| 1| 1| 937| 0.00| 0.21| 0.16|
 , S, | 1| 1| 50| 0.00| 0.06| 0.04|
 , S, N| 1| 1| 4| 0.00| 0.00| 0.00|
 , S, S| 1| 1| 5| 0.00| 0.00| 0.00|
 , S, Y| 1| 1| 12| 0.00| 0.50| 0.42|
 , Y| 1| 1| 8153| 0.00| 0.89| 0.78|
 N| 1| 1| 4202889| 0.09| 0.36| 0.31|
 N, | 1| 1| 28| 0.00| 0.25| 0.25|
 N, N| 1| 1| 203| 0.00| 0.18| 0.16|
 N, NULL| 1| 1| 98| 0.00| 0.20| 0.16|
 N, S| 1| 1| 4| 0.00| 0.25| 0.25|
 N, S, Y| 1| 1| 1| 0.00| 1.00| 1.00|
 N, Y| 1| 1| 50| 0.00| 0.92| 0.86|
 NULL| 1| 1|15377167| 0.31| 0.61| 0.56|
 NULL, | 1| 1| 5294| 0.00| 0.24| 0.21|
 NULL, , | 1| 1| 229| 0.00| 0.06| 0.05|
 NULL, , N| 1| 1| 56| 0.00| 0.00| 0.00|
 NULL, , NULL| 1| 1| 81| 0.00| 0.09| 0.05|
 NULL, , S| 1| 1| 4| 0.00| 0.00| 0.00|
 NULL, , Y| 1| 1| 59| 0.00| 0.80| 0.64|
 NULL, N| 1| 1| 25356| 0.00| 0.21| 0.18|
 NULL, NULL| 1| 1| 150012| 0.00| 0.36| 0.32|
 NULL, NULL, | 1| 1| 568| 0.00| 0.09| 0.07|
 NULL, NULL, N| 1| 1| 10984| 0.00| 0.05| 0.04|
 NULL, NULL, NULL| 1| 1| 52886| 0.00| 0.10| 0.08|
 NULL, NULL, NULL, N| 1| 1| 4| 0.00| 0.25| 0.25|
NULL, NULL, NULL,...| 1| 1| 8| 0.00| 0.00| 0.00|
 NULL, NULL, NULL, Y| 1| 1| 2| 0.00| 0.50| 0.50|
 NULL, NULL, S| 1| 1| 642| 0.00| 0.11| 0.09|
 NULL, NULL, S, Y| 1| 1| 1| 0.00| 0.00| 0.00|
 NULL, NULL, Y| 1| 1| 9144| 0.00| 0.75| 0.60|
 NULL, S| 1| 1| 4904| 0.00| 0.16| 0.14|
 NULL, S, | 1| 1| 35| 0.00| 0.11| 0.09|
 NULL, S, N| 1| 1| 175| 0.00| 0.05| 0.03|
 NULL, S, NULL| 1| 1| 939| 0.00| 0.09| 0.07|
 NULL, S, S| 1| 1| 70| 0.00| 0.06| 0.03|
 NULL, S, S, N| 1| 1| 1| 0.00| 0.00| 0.00|
 NULL, S, S, NULL| 1| 1| 1| 0.00| 0.00| 0.00|
 NULL, S, Y| 1| 1| 166| 0.00| 0.70| 0.48|
 NULL, Y| 1| 1| 54567| 0.00| 0.87| 0.77|
 S| 1| 1| 1603374| 0.03| 0.42| 0.37|
 S, | 1| 1| 206615| 0.00| 0.19| 0.15|
 S, , | 1| 1| 162| 0.00| 0.15| 0.14|
 S, , , | 1| 1| 3| 0.00| 0.00| 0.00|
 S, , N| 1| 1| 23| 0.00| 0.09| 0.04|
 S, , NULL| 1| 1| 1| 0.00| 0.00| 0.00|
 S, , NULL, | 1| 1| 1| 0.00| 0.00| 0.00|
 S, , S| 1| 1| 1| 0.00| 0.00| 0.00|
 S, , S, | 1| 1| 1| 0.00| 1.00| 1.00|
 S, , Y| 1| 1| 19| 0.00| 0.89| 0.79|
 S, N| 1| 1| 108165| 0.00| 0.12| 0.10|
 S, N, N| 1| 1| 1| 0.00| 1.00| 1.00|
 S, N, NULL| 1| 1| 1| 0.00| 1.00| 1.00|
 S, NULL| 1| 1| 270587| 0.01| 0.26| 0.22|
 S, NULL, | 1| 1| 37| 0.00| 0.08| 0.05|
 S, NULL, , | 1| 1| 1| 0.00| 0.00| 0.00|
 S, NULL, N| 1| 1| 217| 0.00| 0.09| 0.07|
 S, NULL, NULL| 1| 1| 1179| 0.00| 0.13| 0.11|
 S, NULL, NULL, | 1| 1| 2| 0.00| 0.00| 0.00|
 S, NULL, NULL, N| 1| 1| 7| 0.00| 0.00| 0.00|
 S, NULL, NULL, NULL| 1| 1| 59| 0.00| 0.05| 0.03|
S, NULL, NULL, NU...| 1| 1| 1| 0.00| 0.00| 0.00|
 S, NULL, NULL, Y| 1| 1| 12| 0.00| 0.58| 0.50|
 S, NULL, S| 1| 1| 9| 0.00| 0.11| 0.00|
 S, NULL, S, | 1| 1| 2| 0.00| 0.00| 0.00|
 S, NULL, S, NULL| 1| 1| 1| 0.00| 0.00| 0.00|
 S, NULL, S, Y| 1| 1| 2| 0.00| 0.50| 0.50|
 S, NULL, Y| 1| 1| 227| 0.00| 0.85| 0.69|
 S, S| 1| 1| 58237| 0.00| 0.21| 0.18|
 S, S, | 1| 1| 2800| 0.00| 0.22| 0.16|
 S, S, , | 1| 1| 3| 0.00| 0.00| 0.00|
 S, S, , N| 1| 1| 1| 0.00| 0.00| 0.00|
 S, S, , Y| 1| 1| 2| 0.00| 0.50| 0.50|
 S, S, N| 1| 1| 1453| 0.00| 0.09| 0.07|
 S, S, NULL| 1| 1| 6052| 0.00| 0.19| 0.16|
 S, S, NULL, | 1| 1| 1| 0.00| 0.00| 0.00|
 S, S, NULL, NULL| 1| 1| 8| 0.00| 0.

In [46]:
df_final\
    .filter(F.col('o_ptnt_response_seq').like("N%"))\
    .groupby('o_ptnt_response_seq')\
    .agg((F.count('*')).alias('count'), ((F.count('*')/df_final.count()).cast("decimal(18,2)")).alias('percentage'))\
    .sort(desc('o_ptnt_response_seq')).show(1000)

+--------------------+-------+----------+
 o_ptnt_response_seq| count|percentage|
+--------------------+-------+----------+
 Y, Y, Y, Y, Y, NULL| 2| 0.00|
 Y, Y, Y, Y, S, NULL| 2| 0.00|
 Y, Y, Y, Y, NULL, Y| 1| 0.00|
 Y, Y, Y, Y, NULL| 57| 0.00|
 Y, Y, Y, Y, N, N, N| 1| 0.00|
 Y, Y, Y, Y, N, N| 3| 0.00|
 Y, Y, Y, Y, N| 16| 0.00|
 Y, Y, Y, S, S, NULL| 1| 0.00|
 Y, Y, Y, S, NULL| 52| 0.00|
 Y, Y, Y, S, N| 13| 0.00|
 Y, Y, Y, NULL, Y, Y| 1| 0.00|
 Y, Y, Y, NULL, Y| 12| 0.00|
 Y, Y, Y, NULL, S| 1| 0.00|
Y, Y, Y, NULL, NU...| 1| 0.00|
Y, Y, Y, NULL, NU...| 1| 0.00|
Y, Y, Y, NULL, NU...| 1| 0.00|
Y, Y, Y, NULL, NU...| 1| 0.00|
 Y, Y, Y, NULL, NULL| 34| 0.00|
 Y, Y, Y, NULL, N| 5| 0.00|
 Y, Y, Y, NULL, , | 1| 0.00|
 Y, Y, Y, NULL, | 4| 0.00|
 Y, Y, Y, NULL| 1115| 0.00|
 Y, Y, Y, N, NULL| 4| 0.00|
 Y, Y, Y, N, N| 36| 0.00|
 Y, Y, Y, N, | 1| 0.00|
 Y, Y, Y, N| 312| 0.00|
 Y, Y, Y, , NULL| 1| 0.00|
 Y, Y, S, Y, Y, NULL| 4| 0.00|
 Y, Y, S, Y, NULL, Y| 1| 0.00|
 Y, Y, S, Y, NULL| 3| 0.00|
 Y, Y, S, S, NULL| 31| 0.00|
 Y, Y, S, S, N, N| 1| 0.00|
 Y, Y, S, S, N| 14| 0.00|
 Y, Y, S, NULL, Y| 3| 0.00|
 Y, Y, S, NULL, NULL| 45| 0.00|
 Y, Y, S, NULL, | 40| 0.00|
 Y, Y, S, NULL| 7462| 0.00|
 Y, Y, S, N, NULL, N| 1| 0.00|
 Y, Y, S, N, NULL| 2| 0.00|
 Y, Y, S, N, N| 194| 0.00|
 Y, Y, S, N| 2815| 0.00|
 Y, Y, S, , NULL| 33| 0.00|
Y, Y, NULL, Y, Y,...| 1| 0.00|
 Y, Y, NULL, Y, Y| 71| 0.00|
 Y, Y, NULL, Y, S, Y| 1| 0.00|
 Y, Y, NULL, Y, NULL| 7| 0.00|
 Y, Y, NULL, Y, N| 1| 0.00|
 Y, Y, NULL, Y, | 1| 0.00|
 Y, Y, NULL, Y| 470| 0.00|
 Y, Y, NULL, S, Y| 4| 0.00|
 Y, Y, NULL, S, S| 2| 0.00|
 Y, Y, NULL, S, NULL| 5| 0.00|
 Y, Y, NULL, S, N| 1| 0.00|
 Y, Y, NULL, S, | 1| 0.00|
 Y, Y, NULL, S| 27| 0.00|
Y, Y, NULL, NULL,...| 4| 0.00|
 Y, Y, NULL, NULL, Y| 7| 0.00|
Y, Y, NULL, NULL,...| 1| 0.00|
Y, Y, NULL, NULL,...| 32| 0.00|
Y, Y, NULL, NULL,...| 3| 0.00|
 Y, Y, NULL, NULL, N| 4| 0.00|
Y, Y, NULL, NULL, , | 1| 0.00|
 Y, Y, NULL, NULL, | 10| 0.00|
 Y, Y, NULL, NULL| 1614| 0.00|
 Y, Y, NULL, N, NULL| 2| 0.00|
 Y, Y, NULL, N, N, N| 1| 0.00|
 Y, Y, NULL, N, N| 25| 0.00|
 Y, Y, NULL, N| 295| 0.00|
 Y, Y, NULL, , Y| 4| 0.00|
 Y, Y, NULL, , NULL| 4| 0.00|
 Y, Y, NULL, , N| 1| 0.00|
 Y, Y, NULL, , | 2| 0.00|
 Y, Y, NULL, | 183| 0.00|
 Y, Y, NULL| 41416| 0.00|
 Y, Y, N, Y, Y| 8| 0.00|
 Y, Y, N, Y| 54| 0.00|
 Y, Y, N, S, NULL| 1| 0.00|
 Y, Y, N, S| 8| 0.00|
 Y, Y, N, NULL, Y| 1| 0.00|
 Y, Y, N, NULL, NULL| 4| 0.00|
 Y, Y, N, NULL, N| 1| 0.00|
 Y, Y, N, NULL| 211| 0.00|
 Y, Y, N, N, Y, Y| 2| 0.00|
 Y, Y, N, N, Y| 8| 0.00|
 Y, Y, N, N, NULL| 22| 0.00|
 Y, Y, N, N, N, NULL| 1| 0.00|
 Y, Y, N, N, N, N, N| 1| 0.00|
 Y, Y, N, N, N, N| 7| 0.00|
 Y, Y, N, N, N| 61| 0.00|
 Y, Y, N, N, | 3| 0.00|
 Y, Y, N, N| 1343| 0.00|
 Y, Y, N, | 44| 0.00|
 Y, Y, N| 12802| 0.00|
 Y, Y, , NULL, Y, Y| 1| 0.00|
 Y, Y, , NULL, N| 2| 0.00|
 Y, Y, , NULL| 105| 0.00|
 Y, Y, , N, N, | 1| 0.00|
 Y, Y, , N, N| 12| 0.00|
 Y, Y, , N, | 1| 0.00|
 Y, Y, , N| 46| 0.00|
 Y, Y, , , N, N| 1| 0.00|
 Y, Y, , , N| 1| 0.00|
Y, S, Y, Y, S, NU...| 1| 0.00|
 Y, S, Y, Y, NULL| 3| 0.00|
 Y, S, Y, NULL, Y, Y| 1| 0.00|
 Y, S, Y, NULL, Y| 2| 0.00|
 Y, S, Y, NULL, N| 1| 0.00|
 Y, S, Y, NULL| 31| 0.00|
 Y, S, Y, N| 1| 0.00|
 Y, S, S, S, NULL| 3| 0.00|
 Y, S, S, NULL, NULL| 11| 0.00|
 Y, S, S, NULL, | 2| 0.00|
 Y, S, S, NULL| 368| 0.00|
 Y, S, S, N, N| 4| 0.00|
 Y, S, S, N| 102| 0.00|
 Y, S, S, , NULL| 1| 0.00|
 Y, S, NULL, Y, Y| 3| 0.00|
 Y, S, NULL, Y| 28| 0.00|
Y, S, NULL, NULL,...| 4| 0.00|
 Y, S, NULL, NULL, | 1| 0.00|
 Y, S, NULL, NULL| 310| 0.00|
 Y, S, NULL, N, NULL| 1| 0.00|
 Y, S, NULL, N, N| 2| 0.00|
 Y, S, NULL, N| 8| 0.00|
 Y, S, NULL, | 297| 0.00|
 Y, S, NULL| 92551| 0.00|
 Y, S, N, Y, Y| 1| 0.00|
 Y, S, N, Y| 2| 0.00|
 Y, S, N, NULL, NULL| 1| 0.00|
 Y, S, N, NULL| 5| 0.00|
 Y, S, N, N, NULL| 1| 0.00|
 Y, S, N, N, N, N| 1| 0.00|
 Y, S, N, N, N| 3| 0.00|
 Y, S, N, N| 1306| 0.00|
 Y, S, N, | 6| 0.00|
 Y, S, N| 46770| 0.00|
 Y, S, , NULL| 307| 0.00|
 Y, S, , N| 3| 0.00|
 Y, NULL, Y, Y, Y, Y| 1| 0.00|
 Y, NULL, 

In [47]:
df_final\
    .filter(F.col('o_ptnt_response_seq').like("N%"))\
    .groupby('o_ptnt_response_seq').count()\
    .sort(desc('o_ptnt_response_seq')).show(1000)

+--------------------+--------+
 o_ptnt_response_seq| count|
+--------------------+--------+
 NULL, Y, Y, Y, Y, Y| 12|
NULL, Y, Y, Y, Y,...| 1|
 NULL, Y, Y, Y, Y| 353|
 NULL, Y, Y, Y, S| 3|
NULL, Y, Y, Y, NU...| 1|
 NULL, Y, Y, Y, NULL| 65|
 NULL, Y, Y, Y, N, N| 1|
 NULL, Y, Y, Y, N| 8|
 NULL, Y, Y, Y| 6001|
 NULL, Y, Y, S, Y, Y| 1|
 NULL, Y, Y, S, Y| 6|
 NULL, Y, Y, S, S| 17|
NULL, Y, Y, S, NU...| 1|
 NULL, Y, Y, S, NULL| 11|
 NULL, Y, Y, S, | 1|
 NULL, Y, Y, S| 236|
NULL, Y, Y, NULL,...| 1|
 NULL, Y, Y, NULL, Y| 15|
NULL, Y, Y, NULL,...| 46|
 NULL, Y, Y, NULL, N| 2|
 NULL, Y, Y, NULL, | 12|
 NULL, Y, Y, NULL| 6029|
 NULL, Y, Y, N, NULL| 1|
 NULL, Y, Y, N, N, Y| 1|
 NULL, Y, Y, N, N| 77|
 NULL, Y, Y, N| 1242|
NULL, Y, Y, , S, ...| 1|
 NULL, Y, Y, , NULL| 26|
 NULL, Y, Y, | 263|
 NULL, Y, Y| 206025|
 NULL, Y, S, Y, Y| 5|
 NULL, Y, S, Y| 89|
 NULL, Y, S, S, N| 1|
 NULL, Y, S, S| 77|
 NULL, Y, S, NULL, | 1|
 NULL, Y, S, NULL| 142|
 NULL, Y, S, N, N| 1|
 NULL, Y, S, N| 30|
 NULL, Y, S, , NULL| 1|
 NULL, Y, S, | 31|
 NULL, Y, S| 3708|
NULL, Y, NULL, Y,...| 1|
 NULL, Y, NULL, Y, Y| 21|
NULL, Y, NULL, Y,...| 13|
 NULL, Y, NULL, Y, N| 2|
 NULL, Y, NULL, Y| 322|
 NULL, Y, NULL, S| 1|
NULL, Y, NULL, NU...| 1|
NULL, Y, NULL, NU...| 3|
NULL, Y, NULL, NU...| 2|
 NULL, Y, NULL, NULL| 288|
 NULL, Y, NULL, N, N| 2|
 NULL, Y, NULL, N| 17|
NULL, Y, NULL, , ...| 4|
 NULL, Y, NULL, | 213|
 NULL, Y, NULL| 87213|
 NULL, Y, N, Y| 5|
 NULL, Y, N, S| 1|
 NULL, Y, N, NULL, N| 3|
 NULL, Y, N, NULL| 8|
 NULL, Y, N, N, N| 4|
 NULL, Y, N, N| 610|
 NULL, Y, N, , | 1|
 NULL, Y, N| 20782|
 NULL, Y, , Y, | 1|
 NULL, Y, , Y| 3|
 NULL, Y, , NULL, | 1|
 NULL, Y, , NULL| 239|
 NULL, Y, , , | 1|
 NULL, Y, , | 10|
 NULL, Y, | 6742|
 NULL, Y| 2254032|
 NULL, S, Y, Y| 371|
 NULL, S, Y, NULL, Y| 1|
 NULL, S, Y, NULL, | 1|
 NULL, S, Y, NULL| 5|
 NULL, S, Y, | 2|
 NULL, S, Y| 6558|
 NULL, S, S, Y, Y| 3|
 NULL, S, S, Y, | 1|
 NULL, S, S, Y| 30|
 NULL, S, S, S, Y| 1|
 NULL, S, S, S, S| 4|
 NULL, S, S, S, N| 1|
 NULL, S, S, S| 103|
NULL, S, S, NULL,...| 1|
 NULL, S, S, NULL| 77|
 NULL, S, S, N, N| 3|
 NULL, S, S, N| 31|
 NULL, S, S, | 11|
 NULL, S, S| 6983|
 NULL, S, NULL, Y| 10|
NULL, S, NULL, S, , | 1|
NULL, S, NULL, NU...| 2|
NULL, S, NULL, NU...| 5|
NULL, S, NULL, NU...| 2|
 NULL, S, NULL, NULL| 113|
NULL, S, NULL, N,...| 1|
 NULL, S, NULL, N, N| 1|
 NULL, S, NULL, N| 3|
 NULL, S, NULL, | 160|
 NULL, S, NULL| 50528|
 NULL, S, N, Y| 1|
 NULL, S, N, NULL| 3|
 NULL, S, N, N, N| 1|
 NULL, S, N, N| 200|
 NULL, S, N| 6634|
 NULL, S, , Y| 5|
 NULL, S, , NULL| 135|
 NULL, S, , N| 1|
 NULL, S, , , | 1|
 NULL, S, , | 24|
 NULL, S, | 3411|
 NULL, S| 254695|
NULL, NULL, Y, Y,...| 13|
NULL, NULL, Y, Y,...| 5|
NULL, NULL, Y, Y,...| 1|
 NULL, NULL, Y, Y, Y| 191|
 NULL, NULL, Y, Y, S| 5|
NULL, NULL, Y, Y,...| 86|
NULL, NULL, Y, Y,...| 5|
 NULL, NULL, Y, Y, N| 28|
NULL, NULL, Y, Y,...| 1|
 NULL, NULL, Y, Y, | 3|
 NULL, NULL, Y, Y| 25168|
 NULL, NULL, Y, S, Y| 2|
NULL, NULL, Y, S,...| 1|
 NULL, NULL, Y, S, N| 1|
 NULL, NULL, Y, S| 26|
NULL, NULL, Y, NU...| 26|
NULL, NULL, Y, NU...| 2|
NULL, NULL, Y, NU...| 5|
 NULL, NULL, Y, NULL| 841|
 NULL, NULL, Y, N, N| 8|
 NULL, NULL, Y, N| 257|
 NULL, NULL, Y, , Y| 3|
NULL, NULL, Y, , ...| 2|
 NULL, NULL, Y, | 23|
 NULL, NULL, Y| 337545|
 NULL, NULL, S, Y, Y| 24|
 NULL, NULL, S, Y| 248|
 NULL, NULL, S, S, Y| 4|
 NULL, NULL, S, S, S| 7|
NULL, NULL, S, S,...| 5|
 NULL, NULL, S, S| 411|
NULL, NULL, S, NU...| 2|
 NULL, NULL, S, NULL| 502|
 NULL, NULL, S, N, N| 9|
 NULL, NULL, S, N| 131|
 NULL, NULL, S, | 11|
 NULL, NULL, S| 33733|
NULL, NULL, NULL,...| 1|
NULL, NULL, NULL,...| 272|
NULL, NULL, NULL,...| 1|
NULL, NULL, NULL,...| 1|
 NULL, NULL, NULL, Y| 2360|
NULL, NULL, NULL,...| 1|
NULL, NULL, NULL,...| 7|
NULL, NULL, NULL,...| 1|
NULL, NULL, NULL,...| 1|
 NULL, NULL, NULL, S| 176|
NULL, NULL, NULL,...| 1|
NULL, NULL, NULL,...| 8|
NULL, NULL, NULL,...| 1|
NULL, NULL, NULL,...| 601|
NULL, NULL, NULL,...| 4|
NULL, NULL, NULL,...| 7|
NULL, NULL, NULL,..

In [48]:
df_final\
    .filter(F.col('less_than_3_attempt_flg') == 1)\
    .filter(F.col('o_rr_attempt_nbr_outreached') == 3)\
    .groupby('first_response', 'last_response','o_ptnt_response_seq','o_rr_attempt_nbr_outreached')\
    .agg(F.count('*').alias('count'), 
         (F.when(F.col('o_rr_attempt_nbr_outreached') == 1, (F.count('*')/48937081).cast("decimal(18,2)"))\
           .when(F.col('o_rr_attempt_nbr_outreached') == 2, (F.count('*')/24571819).cast("decimal(18,2)"))\
           .when(F.col('o_rr_attempt_nbr_outreached') == 3, (F.count('*')/9893367).cast("decimal(18,2)"))).alias('percentage'), 
         (F.avg('valid_fill_flg').cast("decimal(18,2)")).alias('valid_fill_rate'),  
         (F.avg('rr_target').cast("decimal(18,2)")).alias('pickup_rate'))\
    .sort('first_response','last_response')\
    .show(1000)

+---------------+------------------+--------------------+---------------------------+-------+----------+---------------+-----------+
 first_response| last_response| o_ptnt_response_seq|o_rr_attempt_nbr_outreached| count|percentage|valid_fill_rate|pickup_rate|
+---------------+------------------+--------------------+---------------------------+-------+----------+---------------+-----------+
 , , | , , | , , | 3| 739559| 0.07| 0.07| 0.06|
 , , , | , , , | , , , | 3| 6| 0.00| 0.00| 0.00|
 , , , N| N| , , , N| 3| 1| 0.00| 0.00| 0.00|
 , , , NULL| NULL| , , , NULL| 3| 2| 0.00| 0.50| 0.50|
 , , , Y| Y| , , , Y| 3| 2| 0.00| 0.50| 0.50|
 , , N| N| , , N| 3| 65639| 0.01| 0.03| 0.02|
 , , N, | , , N, | , , N, | 3| 1| 0.00| 0.00| 0.00|
 , , N, N| N| , , N, N| 3| 24| 0.00| 0.00| 0.00|
 , , NULL| NULL| , , NULL| 3| 818| 0.00| 0.55| 0.49|
 , , S| S| , , S| 3| 4731| 0.00| 0.10| 0.08|
 , , S, | , , S, | , , S, | 3| 12| 0.00| 0.25| 0.25|
 , , S, S| S| , , S, S| 3| 1| 0.00| 0.00| 0.00|
 , , S, Y| Y| , , S, Y| 3| 1| 0.00| 0.00| 0.00|
 , , Y| Y| , , Y| 3| 44839| 0.00| 0.71| 0.53|
 , , Y, Y| Y| , , Y, Y| 3| 43| 0.00| 0.84| 0.67|
 , N, | , N, | , N, | 3| 56271| 0.01| 0.04| 0.03|
 , N, , | , N, , | , N, , | 3| 1| 0.00| 0.00| 0.00|
 , N, N| N| , N, N| 3| 71543| 0.01| 0.08| 0.06|
 , N, N, | , N, N, | , N, N, | 3| 10| 0.00| 0.10| 0.00|
 , N, N, N| N| , N, N, N| 3| 18| 0.00| 0.00| 0.00|
 , N, N, Y| Y| , N, N, Y| 3| 1| 0.00| 1.00| 1.00|
 , N, NULL| NULL| , N, NULL| 3| 1| 0.00| 0.00| 0.00|
 , N, S| S| , N, S| 3| 365| 0.00| 0.06| 0.05|
 , N, Y| Y| , N, Y| 3| 5495| 0.00| 0.77| 0.61|
 , N, Y, Y| Y| , N, Y, Y| 3| 1| 0.00| 1.00| 1.00|
 , NULL, | , NULL, | , NULL, | 3| 38889| 0.00| 0.14| 0.12|
 , NULL, , | , NULL, , | , NULL, , | 3| 27| 0.00| 0.11| 0.07|
 , NULL, , N| N| , NULL, , N| 3| 5| 0.00| 0.00| 0.00|
 , NULL, , Y| Y| , NULL, , Y| 3| 3| 0.00| 1.00| 0.67|
 , NULL, N| N| , NULL, N| 3| 5430| 0.00| 0.26| 0.22|
 , NULL, NULL| NULL| , NULL, NULL| 3| 27178| 0.00| 0.69| 0.60|
 , NULL, NULL, | , NULL, NULL, | , NULL, NULL, | 3| 15| 0.00| 0.07| 0.00|
, NULL, NULL, Y| Y| , NULL, NULL, Y| 3| 4| 0.00| 0.75| 0.75|
 , NULL, S| S| , NULL, S| 3| 1570| 0.00| 0.23| 0.18|
 , NULL, S, | , NULL, S, | , NULL, S, | 3| 38| 0.00| 0.16| 0.13|
 , NULL, S, N| N| , NULL, S, N| 3| 1| 0.00| 0.00| 0.00|
 , NULL, S, Y| Y| , NULL, S, Y| 3| 3| 0.00| 0.67| 0.67|
 , NULL, Y| Y| , NULL, Y| 3| 11012| 0.00| 0.90| 0.79|
 , S, | , S, | , S, | 3| 954| 0.00| 0.13| 0.11|
 , S, , | , S, , | , S, , | 3| 12| 0.00| 0.17| 0.17|
 , S, , N| N| , S, , N| 3| 1| 0.00| 0.00| 0.00|
 , S, , NULL| NULL| , S, , NULL| 3| 5| 0.00| 0.00| 0.00|
 , S, N| N| , S, N| 3| 72| 0.00| 0.03| 0.03|
 , S, N, N| N| , S, N, N| 3| 38| 0.00| 0.05| 0.05|
 , S, NULL, | , S, NULL, | , S, NULL, | 3| 3| 0.00| 0.00| 0.00|
 , S, S| S| , S, S| 3| 1038| 0.00| 0.16| 0.13|
 , S, S, S| S| , S, S, S| 3| 5| 0.00| 0.00| 0.00|
 , S, Y| Y| , S, Y| 3| 107| 0.00| 0.61| 0.41|
 , S, Y, | , S, Y, | , S, Y, | 3| 2| 0.00| 0.50| 0.50|
 , S, Y, Y| Y| , S, Y, Y| 3| 61| 0.00| 0.52| 0.31|
 , Y, | , Y, | , Y, | 3| 23570| 0.00| 0.08| 0.07|
 , Y, N| N| , Y, N| 3| 4136| 0.00| 0.04| 0.03|
 , Y, S| S| , Y, S| 3| 989| 0.00| 0.09| 0.07|
 , Y, S, | , Y, S, | , Y, S, | 3| 16| 0.00| 0.00| 0.00|
 , Y, S, N| N| , Y, S, N| 3| 2| 0.00| 0.00| 0.00|
 , Y, S, Y| Y| , Y, S, Y| 3| 6| 0.00| 0.17| 0.17|
 , Y, Y| Y| , Y, Y| 3| 63477| 0.01| 0.82| 0.68|
 , Y, Y, | , Y, Y, | , Y, Y, | 3| 9| 0.00| 0.00| 0.00|
 , Y, Y, Y| Y| , Y, Y, Y| 3| 13| 0.00| 0.46| 0.38|
 N| N| N, S, N| 3| 742| 0.00| 0.05| 0.04|
 N| N| N, S, , N| 3| 1| 0.00| 0.00| 0.00|
 N| N| N, NULL, NULL, N| 3| 1| 0.00| 0.00| 0.00|
 N| N| N, , N| 3| 33372| 0.00| 0.03| 0.02|
 N| N| N, Y, N| 3| 10172| 0.00| 0.04| 0.03|
 N| N| N, N, , N| 3| 11| 0.00| 0.09| 0.09|
 N| N| N, NULL, N, N| 3| 38| 0.00| 0.16| 0.08|
 N| N| N, N, N| 3| 988792| 0.10| 0.06| 0.05|
 N| N| N, S, NULL, N| 3| 1| 0.00| 0.00| 0.00|
 N| N| N, N, N, N| 3| 12| 0.00| 0.00| 0.00|
 N| N| N, N, S, N| 3| 231| 0.00| 0.03| 0.02|
 N| N| N, Y, S, N| 3| 10| 0.00| 0.00| 0.00|
 N| N

In [49]:
print df_final_clean.count(), 
print df_final_clean.select('o_store_nbr', 'o_rx_nbr', 'o_rx_fill_nbr').distinct().count(), 
print df_final_clean.select('store_nbr', 'rx_nbr', 'rx_fill_nbr').distinct().count()

80819563 80819563 44950085

In [50]:
df_final\
    .groupby('less_than_3_attempt_flg')\
    .agg((F.count('*')).alias('count'), ((F.count('*')/df_final.count()).cast("decimal(18,2)")).alias('percentage'), 
         (F.avg('valid_fill_flg').cast("decimal(18,2)")).alias('valid_fill_rate'),  
         (F.avg('rr_target').cast("decimal(18,2)")).alias('pickup_rate'))\
    .sort(desc('less_than_3_attempt_flg')).show()

+-----------------------+--------+----------+---------------+-----------+
less_than_3_attempt_flg| count|percentage|valid_fill_rate|pickup_rate|
+-----------------------+--------+----------+---------------+-----------+
 1|83402267| 0.99| 0.56| 0.50|
 0| 454976| 0.01| 0.19| 0.15|
+-----------------------+--------+----------+---------------+-----------+

In [51]:
df_final.agg(F.min('o_run_dt'),F.max('o_run_dt')).show()

+-------------------+-------------------+
 min(o_run_dt)| max(o_run_dt)|
+-------------------+-------------------+
2018-06-01 00:00:00|2019-01-31 00:00:00|
+-------------------+-------------------+

In [52]:
df_final_clean.groupby('o_rr_attempt_nbr_outreached')\
        .agg(F.count('*').alias('count'),
             (F.count('*')/df_final_clean.count()).cast("decimal(18,2)").alias('percentage'),
             (F.avg('valid_fill_flg').cast("decimal(18,2)")).alias('valid_fill_rate'),  
             (F.avg('rr_target').cast("decimal(18,2)")).alias('pickup_rate'))\
        .sort('o_rr_attempt_nbr_outreached')\
        .show()

+---------------------------+--------+----------+---------------+-----------+
o_rr_attempt_nbr_outreached| count|percentage|valid_fill_rate|pickup_rate|
+---------------------------+--------+----------+---------------+-----------+
 1|48897292| 0.61| 0.70| 0.64|
 2|22991606| 0.28| 0.43| 0.37|
 3| 8930665| 0.11| 0.11| 0.09|
+---------------------------+--------+----------+---------------+-----------+

In [53]:
df_final\
    .filter(F.col('less_than_3_attempt_flg') == 1)\
    .filter(F.col('o_rr_attempt_nbr_outreached') == 1)\
    .groupby('o_ptnt_response_seq','less_than_3_attempt_flg', 'o_rr_attempt_nbr_outreached')\
    .agg(F.count('*').alias('count'), 
         (F.when(F.col('o_rr_attempt_nbr_outreached') == 1, (F.count('*')/48937081).cast("decimal(18,2)"))\
           .when(F.col('o_rr_attempt_nbr_outreached') == 2, (F.count('*')/24571819).cast("decimal(18,2)"))\
           .when(F.col('o_rr_attempt_nbr_outreached') == 3, (F.count('*')/9893367).cast("decimal(18,2)"))).alias('percentage'), 
         (F.avg('valid_fill_flg').cast("decimal(18,2)")).alias('valid_fill_rate'),  
         (F.avg('rr_target').cast("decimal(18,2)")).alias('pickup_rate'))\
    .sort('o_ptnt_response_seq').show(1000)

+--------------------+-----------------------+---------------------------+--------+----------+---------------+-----------+
 o_ptnt_response_seq|less_than_3_attempt_flg|o_rr_attempt_nbr_outreached| count|percentage|valid_fill_rate|pickup_rate|
+--------------------+-----------------------+---------------------------+--------+----------+---------------+-----------+
 | 1| 1| 6602451| 0.13| 0.55| 0.48|
 , | 1| 1| 33718| 0.00| 0.34| 0.29|
 , , | 1| 1| 2529| 0.00| 0.08| 0.06|
 , , N| 1| 1| 484| 0.00| 0.05| 0.03|
 , , S| 1| 1| 42| 0.00| 0.12| 0.10|
 , , Y| 1| 1| 346| 0.00| 0.75| 0.55|
 , N| 1| 1| 4225| 0.00| 0.25| 0.21|
 , N, | 1| 1| 1| 0.00| 0.00| 0.00|
 , NULL| 1| 1| 368| 0.00| 0.57| 0.51|
 , NULL, | 1| 1| 572| 0.00| 0.06| 0.05|
 , NULL, N| 1| 1| 17| 0.00| 0.06| 0.00|
 , NULL, S| 1| 1| 2| 0.00| 0.00| 0.00|
 , NULL, Y| 1| 1| 17| 0.00| 0.82| 0.59|
 , S| 1| 1| 937| 0.00| 0.21| 0.16|
 , S, | 1| 1| 50| 0.00| 0.06| 0.04|
 , S, N| 1| 1| 4| 0.00| 0.00| 0.00|
 , S, S| 1| 1| 5| 0.00| 0.00| 0.00|
 , S, Y| 1| 1| 12| 0.00| 0.50| 0.42|
 , Y| 1| 1| 8153| 0.00| 0.89| 0.78|
 N| 1| 1| 4202889| 0.09| 0.36| 0.31|
 N, | 1| 1| 28| 0.00| 0.25| 0.25|
 N, N| 1| 1| 203| 0.00| 0.18| 0.16|
 N, NULL| 1| 1| 98| 0.00| 0.20| 0.16|
 N, S| 1| 1| 4| 0.00| 0.25| 0.25|
 N, S, Y| 1| 1| 1| 0.00| 1.00| 1.00|
 N, Y| 1| 1| 50| 0.00| 0.92| 0.86|
 NULL| 1| 1|15377167| 0.31| 0.61| 0.56|
 NULL, | 1| 1| 5294| 0.00| 0.24| 0.21|
 NULL, , | 1| 1| 229| 0.00| 0.06| 0.05|
 NULL, , N| 1| 1| 56| 0.00| 0.00| 0.00|
 NULL, , NULL| 1| 1| 81| 0.00| 0.09| 0.05|
 NULL, , S| 1| 1| 4| 0.00| 0.00| 0.00|
 NULL, , Y| 1| 1| 59| 0.00| 0.80| 0.64|
 NULL, N| 1| 1| 25356| 0.00| 0.21| 0.18|
 NULL, NULL| 1| 1| 150012| 0.00| 0.36| 0.32|
 NULL, NULL, | 1| 1| 568| 0.00| 0.09| 0.07|
 NULL, NULL, N| 1| 1| 10984| 0.00| 0.05| 0.04|
 NULL, NULL, NULL| 1| 1| 52886| 0.00| 0.10| 0.08|
 NULL, NULL, NULL, N| 1| 1| 4| 0.00| 0.25| 0.25|
NULL, NULL, NULL,...| 1| 1| 8| 0.00| 0.00| 0.00|
 NULL, NULL, NULL, Y| 1| 1| 2| 0.00| 0.50| 0.50|
 NULL, NULL, S| 1| 1| 642| 0.00| 0.11| 0.09|
 NULL, NULL, S, Y| 1| 1| 1| 0.00| 0.00| 0.00|
 NULL, NULL, Y| 1| 1| 9144| 0.00| 0.75| 0.60|
 NULL, S| 1| 1| 4904| 0.00| 0.16| 0.14|
 NULL, S, | 1| 1| 35| 0.00| 0.11| 0.09|
 NULL, S, N| 1| 1| 175| 0.00| 0.05| 0.03|
 NULL, S, NULL| 1| 1| 939| 0.00| 0.09| 0.07|
 NULL, S, S| 1| 1| 70| 0.00| 0.06| 0.03|
 NULL, S, S, N| 1| 1| 1| 0.00| 0.00| 0.00|
 NULL, S, S, NULL| 1| 1| 1| 0.00| 0.00| 0.00|
 NULL, S, Y| 1| 1| 166| 0.00| 0.70| 0.48|
 NULL, Y| 1| 1| 54567| 0.00| 0.87| 0.77|
 S| 1| 1| 1603374| 0.03| 0.42| 0.37|
 S, | 1| 1| 206615| 0.00| 0.19| 0.15|
 S, , | 1| 1| 162| 0.00| 0.15| 0.14|
 S, , , | 1| 1| 3| 0.00| 0.00| 0.00|
 S, , N| 1| 1| 23| 0.00| 0.09| 0.04|
 S, , NULL| 1| 1| 1| 0.00| 0.00| 0.00|
 S, , NULL, | 1| 1| 1| 0.00| 0.00| 0.00|
 S, , S| 1| 1| 1| 0.00| 0.00| 0.00|
 S, , S, | 1| 1| 1| 0.00| 1.00| 1.00|
 S, , Y| 1| 1| 19| 0.00| 0.89| 0.79|
 S, N| 1| 1| 108165| 0.00| 0.12| 0.10|
 S, N, N| 1| 1| 1| 0.00| 1.00| 1.00|
 S, N, NULL| 1| 1| 1| 0.00| 1.00| 1.00|
 S, NULL| 1| 1| 270587| 0.01| 0.26| 0.22|
 S, NULL, | 1| 1| 37| 0.00| 0.08| 0.05|
 S, NULL, , | 1| 1| 1| 0.00| 0.00| 0.00|
 S, NULL, N| 1| 1| 217| 0.00| 0.09| 0.07|
 S, NULL, NULL| 1| 1| 1179| 0.00| 0.13| 0.11|
 S, NULL, NULL, | 1| 1| 2| 0.00| 0.00| 0.00|
 S, NULL, NULL, N| 1| 1| 7| 0.00| 0.00| 0.00|
 S, NULL, NULL, NULL| 1| 1| 59| 0.00| 0.05| 0.03|
S, NULL, NULL, NU...| 1| 1| 1| 0.00| 0.00| 0.00|
 S, NULL, NULL, Y| 1| 1| 12| 0.00| 0.58| 0.50|
 S, NULL, S| 1| 1| 9| 0.00| 0.11| 0.00|
 S, NULL, S, | 1| 1| 2| 0.00| 0.00| 0.00|
 S, NULL, S, NULL| 1| 1| 1| 0.00| 0.00| 0.00|
 S, NULL, S, Y| 1| 1| 2| 0.00| 0.50| 0.50|
 S, NULL, Y| 1| 1| 227| 0.00| 0.85| 0.69|
 S, S| 1| 1| 58237| 0.00| 0.21| 0.18|
 S, S, | 1| 1| 2800| 0.00| 0.22| 0.16|
 S, S, , | 1| 1| 3| 0.00| 0.00| 0.00|
 S, S, , N| 1| 1| 1| 0.00| 0.00| 0.00|
 S, S, , Y| 1| 1| 2| 0.00| 0.50| 0.50|
 S, S, N| 1| 1| 1453| 0.00| 0.09| 0.07|
 S, S, NULL| 1| 1| 6052| 0.00| 0.19| 0.16|
 S, S, NULL, | 1| 1| 1| 0.00| 0.00| 0.00|
 S, S, NULL, NULL| 1| 1| 8| 0.00| 0.

In [54]:
dbutils.fs.ls("dbfs:/prod/rxperso/pyperso-1.3-py2.7.egg")

Out[ 3 ]: [FileInfo(path=u'dbfs:/prod/rxperso/pyperso-1.3-py2.7.egg', name=u'pyperso-1.3-py2.7.egg', size=715663L)]